In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')
sns.set_context('talk')

In [2]:
df_costs = pd.read_csv("sim_data_full_with_costs_corrected.csv")
df_costs = df_costs[["archetype", "transmission_level", "scenario_number", "mean", "std", "mean_iptsc", "std_iptsc"]]
df_costs.rename(columns={"mean": "cost_mean", "std": "cost_std", "mean_iptsc": "cost_IPTsc_mean", "std_iptsc": "cost_IPTsc_std"}, inplace=True)

df_raw = pd.read_csv("sim_data_raw.csv")
df_raw = df_raw[["archetype", "transmission_level", "scenario_number", "clinical_incidence_all"]]
df_raw["cases"] = df_raw["clinical_incidence_all"]*5000
# Get mean and stdev of cases
df_cases = df_raw.groupby(["archetype", "transmission_level", "scenario_number"]).agg({"cases": ["mean", "std"]}).reset_index()
df = pd.merge(df_costs, df_cases, on=["archetype", "transmission_level", "scenario_number"])

df_scenarios = pd.read_csv("../../run_sims/Assets/scenario_master_list.csv")
df = pd.merge(df, df_scenarios[["archetype", "scenario_number", "comments"]], on=["archetype", "scenario_number"])

df.rename(columns={("cases", "mean"): "cases_mean",
                   ("cases", "std"): "cases_std"}, inplace=True)
df

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\3052325694.py:10: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  df = pd.merge(df_costs, df_cases, on=["archetype", "transmission_level", "scenario_number"])
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\3052325694.py:10: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df = pd.merge(df_costs, df_cases, on=["archetype", "transmission_level", "scenario_number"])


,archetype,transmission_level,scenario_number,cost_mean,cost_std,cost_IPTsc_mean,cost_IPTsc_std,cases_mean,cases_std,comments
0,Central,0.05,0,16107.653428,2112.155014,0.000000,0.000000,6029.420900,170.098339,No IPTsc
1,Central,0.10,0,16515.819897,2174.179624,0.000000,0.000000,5939.191469,153.868286,No IPTsc
2,Central,0.20,0,21151.487610,2955.514287,0.000000,0.000000,6684.382831,103.193884,No IPTsc
3,Central,0.30,0,24133.977951,3471.043854,0.000000,0.000000,6975.190576,88.880312,No IPTsc
4,Central,0.40,0,27399.536886,4047.100109,0.000000,0.000000,7159.926498,83.860444,No IPTsc
...,...,...,...,...,...,...,...,...,...,...
110,Southern,0.05,9,31497.642850,1577.132396,24433.589585,1395.134212,1085.255048,382.213400,DP month
111,Southern,0.10,9,33669.009610,1685.831198,24961.744690,1422.768009,1795.228737,295.557103,DP month
112,Southern,0.20,9,36763.521875,1988.693654,24593.254650,1410.140965,2567.773054,150.665002,DP month
113,Southern,0.30,9,42242.570439,2623.540358,25484.615916,1458.115294,3593.362750,107.524129,DP month


In [3]:
# Convert costs to costs per thousand, and cases to cases per thousand. This is done by dividing by 5 since the base population is 5000 in the simulation
df["cost_mean"] = df["cost_mean"] / 5
df["cost_std"] = df["cost_std"] / 5
df["cost_IPTsc_mean"] = df["cost_IPTsc_mean"] / 5
df["cost_IPTsc_std"] = df["cost_IPTsc_std"] / 5
df["cases_mean"] = df["cases_mean"] / 5
df["cases_std"] = df["cases_std"] / 5
df

,archetype,transmission_level,scenario_number,cost_mean,cost_std,cost_IPTsc_mean,cost_IPTsc_std,cases_mean,cases_std,comments
0,Central,0.05,0,3221.530686,422.431003,0.000000,0.000000,1205.884180,34.019668,No IPTsc
1,Central,0.10,0,3303.163979,434.835925,0.000000,0.000000,1187.838294,30.773657,No IPTsc
2,Central,0.20,0,4230.297522,591.102857,0.000000,0.000000,1336.876566,20.638777,No IPTsc
3,Central,0.30,0,4826.795590,694.208771,0.000000,0.000000,1395.038115,17.776062,No IPTsc
4,Central,0.40,0,5479.907377,809.420022,0.000000,0.000000,1431.985300,16.772089,No IPTsc
...,...,...,...,...,...,...,...,...,...,...
110,Southern,0.05,9,6299.528570,315.426479,4886.717917,279.026842,217.051010,76.442680,DP month
111,Southern,0.10,9,6733.801922,337.166240,4992.348938,284.553602,359.045747,59.111421,DP month
112,Southern,0.20,9,7352.704375,397.738731,4918.650930,282.028193,513.554611,30.133000,DP month
113,Southern,0.30,9,8448.514088,524.708072,5096.923183,291.623059,718.672550,21.504826,DP month


In [4]:
def generate_league_table(df):
    # df: dataframe for single archetype and single transmission level
    # rows are scenarios ordered by descending cases
    # columns are cases, costs, incremental costs, incremental cases, cost-effectiveness ratio
    
    df_table = df.copy()
    
    # Set index to scenario number
    df_table = df_table.set_index("scenario_number")
    
    # Sort by descending cases
    df_table = df_table.sort_values("cases_mean", ascending=False)
    
    df_table["incremental_cases_averted_mean"] = -df_table["cases_mean"].diff()
    df_table["incremental_cases_averted_std"] = np.sqrt(df_table["cases_std"]**2 + df_table["cases_std"].shift(1)**2)
    df_table["incremental_cases_averted_low"] = df_table["incremental_cases_averted_mean"] - 1.96*df_table["incremental_cases_averted_std"]
    df_table["incremental_cases_averted_high"] = df_table["incremental_cases_averted_mean"] + 1.96*df_table["incremental_cases_averted_std"]
    
    df_table["incremental_cost_mean"] = df_table["cost_mean"].diff()
    df_table["incremental_cost_std"] = np.sqrt(df_table["cost_std"]**2 + df_table["cost_std"].shift(1)**2)
    df_table["incremental_cost_low"] = df_table["incremental_cost_mean"] - 1.96*df_table["incremental_cost_std"]
    df_table["incremental_cost_high"] = df_table["incremental_cost_mean"] + 1.96*df_table["incremental_cost_std"]
    
    df_table["incremental_cost_IPTsc_mean"] = df_table["cost_IPTsc_mean"].diff()
    df_table["incremental_cost_IPTsc_std"] = np.sqrt(df_table["cost_IPTsc_std"]**2 + df_table["cost_IPTsc_std"].shift(1)**2)
    df_table["incremental_cost_IPTsc_low"] = df_table["incremental_cost_IPTsc_mean"] - 1.96*df_table["incremental_cost_IPTsc_std"]
    df_table["incremental_cost_IPTsc_high"] = df_table["incremental_cost_IPTsc_mean"] + 1.96*df_table["incremental_cost_IPTsc_std"]

    # Sample incremental costs and cases to get ICER range: 
    def _sample_icers(ico_mean, ico_std, ica_mean, ica_std, n=100000):
        ico = np.random.normal(ico_mean, ico_std, n)
        ica = np.random.normal(ica_mean, ica_std, n)
        return ico / ica
    
    df_table["icer_mean"] = df_table["incremental_cost_mean"] / df_table["incremental_cases_averted_mean"]
    df_table["icer_low"] = np.nan
    df_table["icer_high"] = np.nan
    for i in range(1,len(df_table)):
        icers = _sample_icers(df_table["incremental_cost_mean"].iloc[i], df_table["incremental_cost_std"].iloc[i], df_table["incremental_cases_averted_mean"].iloc[i], df_table["incremental_cases_averted_std"].iloc[i])
        df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
        df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)

    df_table["icer_IPTsc_mean"] = df_table["incremental_cost_IPTsc_mean"] / df_table["incremental_cases_averted_mean"]
    df_table["icer_IPTsc_low"] = np.nan
    df_table["icer_IPTsc_high"] = np.nan
    for i in range(1,len(df_table)):
        icers = _sample_icers(df_table["incremental_cost_IPTsc_mean"].iloc[i], df_table["incremental_cost_IPTsc_std"].iloc[i], df_table["incremental_cases_averted_mean"].iloc[i], df_table["incremental_cases_averted_std"].iloc[i])
        df_table["icer_IPTsc_low"].iloc[i] = np.percentile(icers, 2.5)
        df_table["icer_IPTsc_high"].iloc[i] = np.percentile(icers, 97.5)

    return df_table[["comments",
                     "archetype",
                     "transmission_level",
                     "cases_mean",
                     "incremental_cases_averted_mean",
                     "incremental_cases_averted_low",
                     "incremental_cases_averted_high",
                     "cost_mean",
                     "incremental_cost_mean",
                     "incremental_cost_low",
                     "incremental_cost_high",
                     "icer_mean",
                     "icer_low",
                     "icer_high",
                     "cost_IPTsc_mean",
                     "incremental_cost_IPTsc_mean",
                     "incremental_cost_IPTsc_low",
                     "incremental_cost_IPTsc_high",
                     "icer_IPTsc_mean",
                     "icer_IPTsc_low",
                     "icer_IPTsc_high"]].reset_index()

In [5]:
df_single = df[(df["archetype"] == "Southern") & (df["transmission_level"] == 0.2)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 7, 8, 9])]
df_table = generate_league_table(df_single)
df_table.to_csv("league_table_southern_raw.csv")
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\1814481486.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\1814481486.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\1814481486.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

,scenario_number,comments,archetype,transmission_level,cases_mean,incremental_cases_averted_mean,incremental_cases_averted_low,incremental_cases_averted_high,cost_mean,incremental_cost_mean,...,icer_mean,icer_low,icer_high,cost_IPTsc_mean,incremental_cost_IPTsc_mean,incremental_cost_IPTsc_low,incremental_cost_IPTsc_high,icer_IPTsc_mean,icer_IPTsc_low,icer_IPTsc_high
0,0,No IPTsc,Southern,0.2,998.606661,NaN,NaN,NaN,3664.479750,NaN,...,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
1,6,ASAQ term,Southern,0.2,939.800109,58.806552,-0.102982,117.716086,3918.859251,254.379501,...,4.325700,-41.526168,62.371288,447.983797,447.983797,326.109889,569.857705,7.617923,-9.029701,47.242875
2,8,DP term,Southern,0.2,747.883366,191.916743,121.396936,262.436550,4230.775705,311.916454,...,1.625270,-4.789920,8.467225,1252.164820,804.181023,628.201707,980.160338,4.190260,2.814920,6.903040
3,7,ASAQ month,Southern,0.2,673.520540,74.362826,-4.634596,153.360249,4485.500148,254.724443,...,3.425427,-30.448549,47.281626,1719.331447,467.166627,-82.785721,1017.118976,6.282260,-15.527849,46.496570
4,9,DP month,Southern,0.2,513.554611,159.965929,78.393939,241.537919,7352.704375,2867.204227,...,17.923843,9.269887,39.025105,4918.650930,3199.319483,2429.973916,3968.665051,20.000006,12.155507,41.831124


In [6]:
df_single = df[(df["archetype"] == "Central") & (df["transmission_level"] == 0.2)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 7, 8, 9])]
df_table = generate_league_table(df_single)
df_table.to_csv("league_table_central_raw.csv")
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\1814481486.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\1814481486.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\1814481486.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

,scenario_number,comments,archetype,transmission_level,cases_mean,incremental_cases_averted_mean,incremental_cases_averted_low,incremental_cases_averted_high,cost_mean,incremental_cost_mean,...,icer_mean,icer_low,icer_high,cost_IPTsc_mean,incremental_cost_IPTsc_mean,incremental_cost_IPTsc_low,incremental_cost_IPTsc_high,icer_IPTsc_mean,icer_IPTsc_low,icer_IPTsc_high
0,0,No IPTsc,Central,0.2,1336.876566,NaN,NaN,NaN,4230.297522,NaN,...,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
1,6,ASAQ term,Central,0.2,1243.182270,93.694296,33.483170,153.905422,4379.377127,149.079605,...,1.591128,-19.237830,24.677334,446.115274,446.115274,324.929077,567.301471,4.761392,2.616029,13.331580
2,8,DP term,Central,0.2,925.518857,317.663413,253.385159,381.941667,4475.992750,96.615623,...,0.304145,-4.026086,4.643178,1244.974376,798.859101,624.189415,973.528788,2.514797,1.854165,3.396738
3,7,ASAQ month,Central,0.2,848.124562,77.394295,-2.044341,156.832931,4391.929037,-84.063713,...,-1.086175,-38.468726,32.553552,1563.487567,318.513191,-181.821556,818.847938,4.115461,-7.826320,30.847138
4,9,DP month,Central,0.2,556.799569,291.324994,197.407807,385.242181,6733.081422,2341.152385,...,8.036222,4.061663,13.922634,4468.641240,2905.153673,2208.276029,3602.031317,9.972209,6.739397,15.534368


In [7]:
df_single = df[(df["archetype"] == "Sahel") & (df["transmission_level"] == 0.2)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 7, 8, 9, 30, 31, 32, 33])]
df_table = generate_league_table(df_single)
df_table.to_csv("league_table_sahel_raw.csv")
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\1814481486.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\1814481486.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\1814481486.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

,scenario_number,comments,archetype,transmission_level,cases_mean,incremental_cases_averted_mean,incremental_cases_averted_low,incremental_cases_averted_high,cost_mean,incremental_cost_mean,...,icer_mean,icer_low,icer_high,cost_IPTsc_mean,incremental_cost_IPTsc_mean,incremental_cost_IPTsc_low,incremental_cost_IPTsc_high,icer_IPTsc_mean,icer_IPTsc_low,icer_IPTsc_high
0,0,No IPTsc,Sahel,0.2,906.949171,NaN,NaN,NaN,4215.096226,NaN,...,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
1,30,u10 SMC - SPAQ,Sahel,0.2,798.546588,108.402583,42.887779,173.917387,4389.542257,174.446031,...,1.609242,-13.120237,18.171565,456.108293,456.108293,193.488045,718.728541,4.207541,1.588749,11.467202
2,6,SPAQ term,Sahel,0.2,793.853632,4.692955,-58.756115,68.142026,4233.259455,-156.282801,...,-33.301575,-254.875512,249.114785,381.814261,-74.294032,-364.623926,216.035862,-15.830970,-65.020257,64.653617
3,32,u10 SMC - DP,Sahel,0.2,736.303414,57.550218,-7.856304,122.956740,4826.913770,593.654315,...,10.315414,-51.521291,98.497414,1025.188960,643.374698,352.048464,934.700933,11.179362,-48.175337,84.903789
4,31,u15 SMC - SPAQ,Sahel,0.2,696.593306,39.710108,-30.962036,110.382252,4538.526471,-288.387299,...,-7.262315,-148.991368,125.458869,859.911344,-165.277616,-725.975654,395.420422,-4.162104,-69.019642,59.061873
5,8,DP term,Sahel,0.2,630.694474,65.898832,-6.203063,138.000727,4665.450893,126.924422,...,1.926050,-39.846975,51.798136,1273.672721,413.761377,-97.820766,925.343520,6.278736,-20.470769,49.602928
6,7,SPAQ month,Sahel,0.2,592.782363,37.912111,-33.754461,109.578684,4438.351761,-227.099132,...,-5.990147,-124.982153,114.894573,1099.839962,-173.832759,-600.367339,252.701821,-4.585151,-62.290917,52.910510
7,33,u15 SMC - DP,Sahel,0.2,565.394500,27.387862,-45.088404,99.864129,5340.961619,902.609858,...,32.956565,-280.406425,317.585054,1934.339836,834.499874,193.730352,1475.269395,30.469697,-258.928188,275.713590
8,9,DP month,Sahel,0.2,485.054379,80.340121,1.499747,159.180495,6809.113440,1468.151821,...,18.274205,-2.717435,115.019141,3776.741810,1842.401975,1192.642709,2492.161240,22.932527,7.458162,138.751537


In [8]:
def generate_league_table_PRETTY(df):
    # df: dataframe for single archetype and single transmission level
    # rows are scenarios ordered by descending cases
    # columns are cases, costs, incremental costs, incremental cases, cost-effectiveness ratio
    
    df_table = df.copy()
    
    # Set index to scenario number
    df_table = df_table.set_index("scenario_number")
    
    # Sort by descending cases
    df_table = df_table.sort_values("cases_mean", ascending=False)
    
    df_table["incremental_cost_mean"] = df_table["cost_mean"].diff()
    df_table["incremental_cost_std"] = np.sqrt(df_table["cost_std"]**2 + df_table["cost_std"].shift(1)**2)
    df_table["incremental_cost_low"] = df_table["incremental_cost_mean"] - 1.96*df_table["incremental_cost_std"]
    df_table["incremental_cost_high"] = df_table["incremental_cost_mean"] + 1.96*df_table["incremental_cost_std"]
    
    df_table["incremental_cost_IPTsc_mean"] = df_table["cost_IPTsc_mean"].diff()
    df_table["incremental_cost_IPTsc_std"] = np.sqrt(df_table["cost_IPTsc_std"]**2 + df_table["cost_IPTsc_std"].shift(1)**2)
    df_table["incremental_cost_IPTsc_low"] = df_table["incremental_cost_IPTsc_mean"] - 1.96*df_table["incremental_cost_IPTsc_std"]
    df_table["incremental_cost_IPTsc_high"] = df_table["incremental_cost_IPTsc_mean"] + 1.96*df_table["incremental_cost_IPTsc_std"]
    
    df_table["incremental_cases_averted_mean"] = -df_table["cases_mean"].diff()
    df_table["incremental_cases_averted_std"] = np.sqrt(df_table["cases_std"]**2 + df_table["cases_std"].shift(1)**2)
    df_table["incremental_cases_averted_low"] = df_table["incremental_cases_averted_mean"] - 1.96*df_table["incremental_cases_averted_std"]
    df_table["incremental_cases_averted_high"] = df_table["incremental_cases_averted_mean"] + 1.96*df_table["incremental_cases_averted_std"]
    

    def _sample_icers(ico_mean, ico_std, ica_mean, ica_std, n=100000):
        ico = np.random.normal(ico_mean, ico_std, n)
        ica = np.random.normal(ica_mean, ica_std, n)
        return ico / ica
    
    df_table["icer_mean"] = df_table["incremental_cost_mean"] / df_table["incremental_cases_averted_mean"]
    df_table["icer_low"] = np.nan
    df_table["icer_high"] = np.nan
    for i in range(1,len(df_table)):
        icers = _sample_icers(df_table["incremental_cost_mean"].iloc[i], df_table["incremental_cost_std"].iloc[i], df_table["incremental_cases_averted_mean"].iloc[i], df_table["incremental_cases_averted_std"].iloc[i])
        df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
        df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)

    df_table["icer_IPTsc_mean"] = df_table["incremental_cost_IPTsc_mean"] / df_table["incremental_cases_averted_mean"]
    df_table["icer_IPTsc_low"] = np.nan
    df_table["icer_IPTsc_high"] = np.nan
    for i in range(1,len(df_table)):
        icers = _sample_icers(df_table["incremental_cost_IPTsc_mean"].iloc[i], df_table["incremental_cost_IPTsc_std"].iloc[i], df_table["incremental_cases_averted_mean"].iloc[i], df_table["incremental_cases_averted_std"].iloc[i])
        df_table["icer_IPTsc_low"].iloc[i] = np.percentile(icers, 2.5)
        df_table["icer_IPTsc_high"].iloc[i] = np.percentile(icers, 97.5)

    # Format things nicely
    df_table["cases_mean"] = df_table["cases_mean"].apply(lambda x: "{:.1f}".format(x))
    df_table["cases_std"] = df_table["cases_std"].apply(lambda x: "{:.1f}".format(x))
    df_table = df_table.reset_index()
    df_table.rename(columns={"comments": "Scenario"}, inplace=True)

    # Make new "Cases" column which has the format: "{mean} ({low} - {high})"
    df_table["cases_low"] = (df_table["cases_mean"].astype(float) - 1.96*df_table["cases_std"].astype(float)).apply(lambda x: "{:.1f}".format(x))
    df_table["cases_high"] = (df_table["cases_mean"].astype(float) + 1.96*df_table["cases_std"].astype(float)).apply(lambda x: "{:.1f}".format(x))
    df_table["cases_mean"] = df_table["cases_mean"].astype(str)
    df_table["Cases"] = df_table["cases_mean"] + " (" + df_table["cases_low"] + " - " + df_table["cases_high"] + ")"
    
    # Same for incremental cases
    df_table["incremental_cases_averted_low"] = df_table["incremental_cases_averted_low"].apply(lambda x: "{:.1f}".format(x))
    df_table["incremental_cases_averted_high"] = df_table["incremental_cases_averted_high"].apply(lambda x: "{:.1f}".format(x))
    df_table["incremental_cases_averted_mean"] = df_table["incremental_cases_averted_mean"].apply(lambda x: "{:.1f}".format(x))
    df_table["Incremental Cases Averted"] = df_table["incremental_cases_averted_mean"] + " (" + df_table["incremental_cases_averted_low"] + " - " + df_table["incremental_cases_averted_high"] + ")"
    
    # HEALTH SYSTEM PERSPECTIVE 
    # Same for costs
    df_table["cost_low"] = (df_table["cost_mean"] - 1.96*df_table["cost_std"]).apply(lambda x: "{:.1f}".format(x))
    df_table["cost_high"] = (df_table["cost_mean"] + 1.96*df_table["cost_std"]).apply(lambda x: "{:.1f}".format(x))
    df_table["cost_mean"] = df_table["cost_mean"].apply(lambda x: "{:.1f}".format(x))
    df_table["Cost (Health System Perspective)"] = df_table["cost_mean"] + " (" + df_table["cost_low"] + " - " + df_table["cost_high"] + ")"
    
    # Same for incremental costs
    df_table["incremental_cost_low"] = df_table["incremental_cost_low"].apply(lambda x: "{:.1f}".format(x))
    df_table["incremental_cost_high"] = df_table["incremental_cost_high"].apply(lambda x: "{:.1f}".format(x)
    )
    df_table["incremental_cost_mean"] = df_table["incremental_cost_mean"].apply(lambda x: "{:.1f}".format(x))
    df_table["Incremental Cost (Health System Perspective)"] = df_table["incremental_cost_mean"] + " (" + df_table["incremental_cost_low"] + " - " + df_table["incremental_cost_high"] + ")"
    
    # Same for ICER
    df_table["icer_low"] = df_table["icer_low"].apply(lambda x: "{:.1f}".format(x))
    df_table["icer_high"] = df_table["icer_high"].apply(lambda x: "{:.1f}".format(x))
    df_table["icer_mean"] = df_table["icer_mean"].apply(lambda x: "{:.1f}".format(x))
    # If low-end of ICER is negative, it's dominant
    df_table["icer_low"] = df_table["icer_low"].apply(lambda x: "Dominant" if float(x) < 0 else x)
    df_table["ICER (Health System Perspective)"] = df_table["icer_mean"] + " (" + df_table["icer_low"] + " - " + df_table["icer_high"] + ")"
    
    
    # FUNDER PERSPECTIVE
    # Same for IPTsc costs
    df_table["cost_IPTsc_low"] = (df_table["cost_IPTsc_mean"] - 1.96*df_table["cost_IPTsc_std"]).apply(lambda x: "{:.1f}".format(x))
    df_table["cost_IPTsc_high"] = (df_table["cost_IPTsc_mean"] + 1.96*df_table["cost_IPTsc_std"]).apply(lambda x: "{:.1f}".format(x))
    df_table["cost_IPTsc_mean"] = df_table["cost_IPTsc_mean"].apply(lambda x: "{:.1f}".format(x))
    df_table["Cost (Funder Perspective)"] = df_table["cost_IPTsc_mean"] + " (" + df_table["cost_IPTsc_low"] + " - " + df_table["cost_IPTsc_high"] + ")"

    # Same for incremental IPTsc costs
    df_table["incremental_cost_IPTsc_low"] = df_table["incremental_cost_IPTsc_low"].apply(lambda x: "{:.1f}".format(x))
    df_table["incremental_cost_IPTsc_high"] = df_table["incremental_cost_IPTsc_high"].apply(lambda x: "{:.1f}".format(x))
    df_table["incremental_cost_IPTsc_mean"] = df_table["incremental_cost_IPTsc_mean"].apply(lambda x: "{:.1f}".format(x))
    df_table["Incremental Cost (Funder Perspective)"] = df_table["incremental_cost_IPTsc_mean"] + " (" + df_table["incremental_cost_IPTsc_low"] + " - " + df_table["incremental_cost_IPTsc_high"] + ")"

    # Same for IPTsc ICER
    df_table["icer_IPTsc_low"] = df_table["icer_IPTsc_low"].apply(lambda x: "{:.1f}".format(x))
    df_table["icer_IPTsc_high"] = df_table["icer_IPTsc_high"].apply(lambda x: "{:.1f}".format(x))
    df_table["icer_IPTsc_mean"] = df_table["icer_IPTsc_mean"].apply(lambda x: "{:.1f}".format(x))
    # If low-end of ICER is negative, it's dominant
    df_table["icer_IPTsc_low"] = df_table["icer_IPTsc_low"].apply(lambda x: "Dominant" if float(x) < 0 else x)
    df_table["ICER (Funder Perspective)"] = df_table["icer_IPTsc_mean"] + " (" + df_table["icer_IPTsc_low"] + " - " + df_table["icer_IPTsc_high"] + ")"

    df_table = df_table[["Scenario",
                         "Cases",
                         "Incremental Cases Averted",
                         "Cost (Funder Perspective)",
                         "Incremental Cost (Funder Perspective)",
                         "ICER (Funder Perspective)",
                         "Cost (Health System Perspective)",
                         "Incremental Cost (Health System Perspective)",
                         "ICER (Health System Perspective)",
                         ]]
    return df_table

In [9]:
df_single = df[(df["archetype"] == "Southern") & (df["transmission_level"] == 0.2)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 7, 8, 9])]
df_table = generate_league_table_PRETTY(df_single)
df_table.to_csv("league_table_southern_pretty.csv")
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,998.6 (959.0 - 1038.2),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),3664.5 (2699.8 - 4629.2),nan (nan - nan),nan (nan - nan)
1,ASAQ term,939.8 (896.3 - 983.3),58.8 (-0.1 - 117.7),448.0 (326.1 - 569.9),448.0 (326.1 - 569.9),7.6 (2.3 - 47.8),3918.9 (3010.8 - 4826.9),254.4 (-1070.5 - 1579.3),4.3 (Dominant - 61.8)
2,DP term,747.9 (692.4 - 803.4),191.9 (121.4 - 262.4),1252.2 (1125.2 - 1379.1),804.2 (628.2 - 980.2),4.2 (2.8 - 6.9),4230.8 (3485.2 - 4976.4),311.9 (-863.1 - 1486.9),1.6 (Dominant - 8.5)
3,ASAQ month,673.5 (617.2 - 729.8),74.4 (-4.6 - 153.4),1719.3 (1184.2 - 2254.4),467.2 (-82.8 - 1017.1),6.3 (Dominant - 46.9),4485.5 (3634.4 - 5336.6),254.7 (-876.8 - 1386.2),3.4 (Dominant - 48.2)
4,DP month,513.6 (454.6 - 572.6),160.0 (78.4 - 241.5),4918.7 (4365.9 - 5471.4),3199.3 (2430.0 - 3968.7),20.0 (12.2 - 42.0),7352.7 (6573.1 - 8132.3),2867.2 (1713.0 - 4021.4),17.9 (9.3 - 39.1)


In [10]:
df_single = df[(df["archetype"] == "Central") & (df["transmission_level"] == 0.2)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 7, 8, 9])]
df_table = generate_league_table_PRETTY(df_single)
df_table.to_csv("league_table_central_pretty.csv")
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,1336.9 (1296.5 - 1377.3),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),4230.3 (3071.7 - 5388.9),nan (nan - nan),nan (nan - nan)
1,ASAQ term,1243.2 (1198.5 - 1287.9),93.7 (33.5 - 153.9),446.1 (324.9 - 567.3),446.1 (324.9 - 567.3),4.8 (2.6 - 13.4),4379.4 (3314.6 - 5444.2),149.1 (-1424.5 - 1722.6),1.6 (Dominant - 24.2)
2,DP term,925.5 (879.2 - 971.8),317.7 (253.4 - 381.9),1245.0 (1119.2 - 1370.8),798.9 (624.2 - 973.5),2.5 (1.9 - 3.4),4476.0 (3646.5 - 5305.4),96.6 (-1253.1 - 1446.3),0.3 (Dominant - 4.7)
3,ASAQ month,848.1 (783.6 - 912.6),77.4 (-2.0 - 156.8),1563.5 (1079.2 - 2047.8),318.5 (-181.8 - 818.8),4.1 (Dominant - 31.4),4391.9 (3547.9 - 5235.9),-84.1 (-1267.4 - 1099.3),-1.1 (Dominant - 31.6)
4,DP month,556.8 (488.6 - 625.0),291.3 (197.4 - 385.2),4468.6 (3967.5 - 4969.8),2905.2 (2208.3 - 3602.0),10.0 (6.7 - 15.6),6733.1 (6032.0 - 7434.2),2341.2 (1243.9 - 3438.4),8.0 (4.0 - 13.9)


In [11]:
df_single = df[(df["archetype"] == "Sahel") & (df["transmission_level"] == 0.2)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 7, 8, 9, 30, 31, 32, 33])]
df_table = generate_league_table_PRETTY(df_single)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,906.9 (861.2 - 952.6),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),4215.1 (3242.5 - 5187.7),nan (nan - nan),nan (nan - nan)
1,u10 SMC - SPAQ,798.5 (751.5 - 845.5),108.4 (42.9 - 173.9),456.1 (193.5 - 718.7),456.1 (193.5 - 718.7),4.2 (1.6 - 11.6),4389.5 (3471.8 - 5307.3),174.4 (-1162.8 - 1511.7),1.6 (Dominant - 18.0)
2,SPAQ term,793.9 (751.2 - 836.6),4.7 (-58.8 - 68.1),381.8 (258.0 - 505.6),-74.3 (-364.6 - 216.0),-15.8 (Dominant - 64.3),4233.3 (3373.6 - 5092.9),-156.3 (-1413.8 - 1101.2),-33.3 (Dominant - 258.4)
3,u10 SMC - DP,736.3 (686.7 - 785.9),57.6 (-7.9 - 123.0),1025.2 (761.5 - 1288.9),643.4 (352.0 - 934.7),11.2 (Dominant - 85.4),4826.9 (3947.4 - 5706.5),593.7 (-636.2 - 1823.5),10.3 (Dominant - 98.0)
4,u15 SMC - SPAQ,696.6 (646.2 - 747.0),39.7 (-31.0 - 110.4),859.9 (365.1 - 1354.7),-165.3 (-726.0 - 395.4),-4.2 (Dominant - 58.5),4538.5 (3600.5 - 5476.5),-288.4 (-1574.2 - 997.5),-7.3 (Dominant - 131.9)
5,DP term,630.7 (579.2 - 682.2),65.9 (-6.2 - 138.0),1273.7 (1143.7 - 1403.6),413.8 (-97.8 - 925.3),6.3 (Dominant - 49.0),4665.5 (3950.3 - 5380.6),126.9 (-1052.6 - 1306.4),1.9 (Dominant - 51.8)
6,SPAQ month,592.8 (543.0 - 642.6),37.9 (-33.8 - 109.6),1099.8 (693.6 - 1506.1),-173.8 (-600.4 - 252.7),-4.6 (Dominant - 55.0),4438.4 (3639.2 - 5237.5),-227.1 (-1299.6 - 845.4),-6.0 (Dominant - 118.2)
7,u15 SMC - DP,565.4 (512.7 - 618.1),27.4 (-45.1 - 99.9),1934.3 (1438.8 - 2429.9),834.5 (193.7 - 1475.3),30.5 (Dominant - 281.6),5341.0 (4476.2 - 6205.7),902.6 (-274.9 - 2080.1),33.0 (Dominant - 319.1)
8,DP month,485.1 (426.5 - 543.7),80.3 (1.5 - 159.2),3776.7 (3356.4 - 4197.0),1842.4 (1192.6 - 2492.2),22.9 (7.3 - 137.9),6809.1 (6084.3 - 7534.0),1468.2 (339.8 - 2596.5),18.3 (Dominant - 115.1)


In [12]:
# Remove dominated scenario 30 - u10 SMC - SPAQ
df_single = df[(df["archetype"] == "Sahel") & (df["transmission_level"] == 0.2)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 7, 8, 9, 31, 32, 33])]
df_table = generate_league_table_PRETTY(df_single)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,906.9 (861.2 - 952.6),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),4215.1 (3242.5 - 5187.7),nan (nan - nan),nan (nan - nan)
1,SPAQ term,793.9 (751.2 - 836.6),113.1 (50.6 - 175.6),381.8 (258.0 - 505.6),381.8 (258.0 - 505.6),3.4 (1.9 - 7.8),4233.3 (3373.6 - 5092.9),18.2 (-1279.9 - 1316.2),0.2 (Dominant - 14.1)
2,u10 SMC - DP,736.3 (686.7 - 785.9),57.6 (-7.9 - 123.0),1025.2 (761.5 - 1288.9),643.4 (352.0 - 934.7),11.2 (Dominant - 83.6),4826.9 (3947.4 - 5706.5),593.7 (-636.2 - 1823.5),10.3 (Dominant - 98.3)
3,u15 SMC - SPAQ,696.6 (646.2 - 747.0),39.7 (-31.0 - 110.4),859.9 (365.1 - 1354.7),-165.3 (-726.0 - 395.4),-4.2 (Dominant - 59.1),4538.5 (3600.5 - 5476.5),-288.4 (-1574.2 - 997.5),-7.3 (Dominant - 126.2)
4,DP term,630.7 (579.2 - 682.2),65.9 (-6.2 - 138.0),1273.7 (1143.7 - 1403.6),413.8 (-97.8 - 925.3),6.3 (Dominant - 49.0),4665.5 (3950.3 - 5380.6),126.9 (-1052.6 - 1306.4),1.9 (Dominant - 51.3)
5,SPAQ month,592.8 (543.0 - 642.6),37.9 (-33.8 - 109.6),1099.8 (693.6 - 1506.1),-173.8 (-600.4 - 252.7),-4.6 (Dominant - 51.9),4438.4 (3639.2 - 5237.5),-227.1 (-1299.6 - 845.4),-6.0 (Dominant - 115.3)
6,u15 SMC - DP,565.4 (512.7 - 618.1),27.4 (-45.1 - 99.9),1934.3 (1438.8 - 2429.9),834.5 (193.7 - 1475.3),30.5 (Dominant - 289.5),5341.0 (4476.2 - 6205.7),902.6 (-274.9 - 2080.1),33.0 (Dominant - 335.4)
7,DP month,485.1 (426.5 - 543.7),80.3 (1.5 - 159.2),3776.7 (3356.4 - 4197.0),1842.4 (1192.6 - 2492.2),22.9 (7.3 - 143.7),6809.1 (6084.3 - 7534.0),1468.2 (339.8 - 2596.5),18.3 (Dominant - 114.3)


In [13]:
# Remove dominated scenario 32 - u10 SMC - DP
df_single = df[(df["archetype"] == "Sahel") & (df["transmission_level"] == 0.2)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 7, 8, 9, 31, 33])]
df_table = generate_league_table_PRETTY(df_single)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,906.9 (861.2 - 952.6),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),4215.1 (3242.5 - 5187.7),nan (nan - nan),nan (nan - nan)
1,SPAQ term,793.9 (751.2 - 836.6),113.1 (50.6 - 175.6),381.8 (258.0 - 505.6),381.8 (258.0 - 505.6),3.4 (1.9 - 7.8),4233.3 (3373.6 - 5092.9),18.2 (-1279.9 - 1316.2),0.2 (Dominant - 14.1)
2,u15 SMC - SPAQ,696.6 (646.2 - 747.0),97.3 (31.3 - 163.3),859.9 (365.1 - 1354.7),478.1 (-32.0 - 988.2),4.9 (Dominant - 17.8),4538.5 (3600.5 - 5476.5),305.3 (-967.1 - 1577.6),3.1 (Dominant - 23.4)
3,DP term,630.7 (579.2 - 682.2),65.9 (-6.2 - 138.0),1273.7 (1143.7 - 1403.6),413.8 (-97.8 - 925.3),6.3 (Dominant - 49.5),4665.5 (3950.3 - 5380.6),126.9 (-1052.6 - 1306.4),1.9 (Dominant - 49.9)
4,SPAQ month,592.8 (543.0 - 642.6),37.9 (-33.8 - 109.6),1099.8 (693.6 - 1506.1),-173.8 (-600.4 - 252.7),-4.6 (Dominant - 51.9),4438.4 (3639.2 - 5237.5),-227.1 (-1299.6 - 845.4),-6.0 (Dominant - 114.6)
5,u15 SMC - DP,565.4 (512.7 - 618.1),27.4 (-45.1 - 99.9),1934.3 (1438.8 - 2429.9),834.5 (193.7 - 1475.3),30.5 (Dominant - 282.6),5341.0 (4476.2 - 6205.7),902.6 (-274.9 - 2080.1),33.0 (Dominant - 317.4)
6,DP month,485.1 (426.5 - 543.7),80.3 (1.5 - 159.2),3776.7 (3356.4 - 4197.0),1842.4 (1192.6 - 2492.2),22.9 (7.5 - 138.0),6809.1 (6084.3 - 7534.0),1468.2 (339.8 - 2596.5),18.3 (Dominant - 115.0)


In [14]:
# Remove dominated scenario 8 - DP term
df_single = df[(df["archetype"] == "Sahel") & (df["transmission_level"] == 0.2)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 7, 9, 31, 33])]
df_table = generate_league_table_PRETTY(df_single)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,906.9 (861.2 - 952.6),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),4215.1 (3242.5 - 5187.7),nan (nan - nan),nan (nan - nan)
1,SPAQ term,793.9 (751.2 - 836.6),113.1 (50.6 - 175.6),381.8 (258.0 - 505.6),381.8 (258.0 - 505.6),3.4 (1.9 - 7.9),4233.3 (3373.6 - 5092.9),18.2 (-1279.9 - 1316.2),0.2 (Dominant - 14.1)
2,u15 SMC - SPAQ,696.6 (646.2 - 747.0),97.3 (31.3 - 163.3),859.9 (365.1 - 1354.7),478.1 (-32.0 - 988.2),4.9 (Dominant - 17.9),4538.5 (3600.5 - 5476.5),305.3 (-967.1 - 1577.6),3.1 (Dominant - 23.3)
3,SPAQ month,592.8 (543.0 - 642.6),103.8 (33.0 - 174.6),1099.8 (693.6 - 1506.1),239.9 (-400.3 - 880.1),2.3 (Dominant - 13.1),4438.4 (3639.2 - 5237.5),-100.2 (-1332.4 - 1132.1),-1.0 (Dominant - 14.3)
4,u15 SMC - DP,565.4 (512.7 - 618.1),27.4 (-45.1 - 99.9),1934.3 (1438.8 - 2429.9),834.5 (193.7 - 1475.3),30.5 (Dominant - 281.7),5341.0 (4476.2 - 6205.7),902.6 (-274.9 - 2080.1),33.0 (Dominant - 310.1)
5,DP month,485.1 (426.5 - 543.7),80.3 (1.5 - 159.2),3776.7 (3356.4 - 4197.0),1842.4 (1192.6 - 2492.2),22.9 (7.0 - 140.3),6809.1 (6084.3 - 7534.0),1468.2 (339.8 - 2596.5),18.3 (Dominant - 111.0)


In [15]:
# Remove dominated scenario 7 - SPAQ month
df_single = df[(df["archetype"] == "Sahel") & (df["transmission_level"] == 0.2)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 9, 31, 33])]
df_table = generate_league_table_PRETTY(df_single)
df_table.to_csv("league_table_sahel_pretty.csv")
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,906.9 (861.2 - 952.6),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),4215.1 (3242.5 - 5187.7),nan (nan - nan),nan (nan - nan)
1,SPAQ term,793.9 (751.2 - 836.6),113.1 (50.6 - 175.6),381.8 (258.0 - 505.6),381.8 (258.0 - 505.6),3.4 (1.9 - 7.8),4233.3 (3373.6 - 5092.9),18.2 (-1279.9 - 1316.2),0.2 (Dominant - 13.9)
2,u15 SMC - SPAQ,696.6 (646.2 - 747.0),97.3 (31.3 - 163.3),859.9 (365.1 - 1354.7),478.1 (-32.0 - 988.2),4.9 (Dominant - 18.1),4538.5 (3600.5 - 5476.5),305.3 (-967.1 - 1577.6),3.1 (Dominant - 23.4)
3,u15 SMC - DP,565.4 (512.7 - 618.1),131.2 (58.3 - 204.1),1934.3 (1438.8 - 2429.9),1074.4 (374.2 - 1774.7),8.2 (2.6 - 20.9),5341.0 (4476.2 - 6205.7),802.4 (-473.3 - 2078.2),6.1 (Dominant - 21.5)
4,DP month,485.1 (426.5 - 543.7),80.3 (1.5 - 159.2),3776.7 (3356.4 - 4197.0),1842.4 (1192.6 - 2492.2),22.9 (7.1 - 141.1),6809.1 (6084.3 - 7534.0),1468.2 (339.8 - 2596.5),18.3 (Dominant - 114.8)


# All league tables for Southern

## 5% pfpr

In [16]:
df_single = df[(df["archetype"] == "Southern") & (df["transmission_level"] == 0.05)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 7, 8, 9])]
df_table = generate_league_table_PRETTY(df_single)
df_table.to_csv("league_tables/league_table_southern_pfpr05_all.csv", index=False)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,663.5 (570.2 - 756.8),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),2351.6 (1816.3 - 2886.8),nan (nan - nan),nan (nan - nan)
1,ASAQ term,586.5 (444.8 - 728.2),77.0 (-92.7 - 246.6),445.6 (324.4 - 566.9),445.6 (324.4 - 566.9),5.8 (Dominant - 57.0),2606.5 (2083.0 - 3129.9),254.9 (-493.8 - 1003.5),3.3 (Dominant - 47.3)
2,DP term,383.0 (211.9 - 554.1),203.5 (-18.7 - 425.6),1243.0 (1115.7 - 1370.3),797.4 (621.6 - 973.2),3.9 (Dominant - 27.7),2992.9 (2560.4 - 3425.4),386.4 (-292.6 - 1065.4),1.9 (Dominant - 16.1)
3,ASAQ month,323.0 (147.2 - 498.8),60.0 (-185.3 - 305.4),1707.8 (1176.9 - 2238.7),464.8 (-81.2 - 1010.7),7.7 (Dominant - 53.8),3311.3 (2663.5 - 3959.2),318.4 (-460.5 - 1097.4),5.3 (Dominant - 47.7)
4,DP month,217.1 (67.4 - 366.8),106.0 (-125.1 - 337.0),4886.7 (4339.8 - 5433.6),3178.9 (2416.8 - 3941.1),30.0 (Dominant - 298.6),6299.5 (5681.3 - 6917.8),2988.2 (2092.7 - 3883.7),28.2 (Dominant - 273.1)


In [17]:
df_single = df[(df["archetype"] == "Southern") & (df["transmission_level"] == 0.05)]
df_single = df_single[df_single["scenario_number"].isin([0, 7, 8, 9])]
df_table = generate_league_table_PRETTY(df_single)
df_table.to_csv("league_tables/league_table_southern_pfpr05_reduced.csv", index=False)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,663.5 (570.2 - 756.8),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),2351.6 (1816.3 - 2886.8),nan (nan - nan),nan (nan - nan)
1,DP term,383.0 (211.9 - 554.1),280.4 (85.6 - 475.3),1243.0 (1115.7 - 1370.3),1243.0 (1115.7 - 1370.3),4.4 (2.6 - 14.0),2992.9 (2560.4 - 3425.4),641.3 (-46.8 - 1329.4),2.3 (Dominant - 8.6)
2,ASAQ month,323.0 (147.2 - 498.8),60.0 (-185.3 - 305.4),1707.8 (1176.9 - 2238.7),464.8 (-81.2 - 1010.7),7.7 (Dominant - 54.8),3311.3 (2663.5 - 3959.2),318.4 (-460.5 - 1097.4),5.3 (Dominant - 47.5)
3,DP month,217.1 (67.4 - 366.8),106.0 (-125.1 - 337.0),4886.7 (4339.8 - 5433.6),3178.9 (2416.8 - 3941.1),30.0 (Dominant - 301.9),6299.5 (5681.3 - 6917.8),2988.2 (2092.7 - 3883.7),28.2 (Dominant - 275.2)


## 10% pfpr

In [18]:
df_single = df[(df["archetype"] == "Southern") & (df["transmission_level"] == 0.1)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 7, 8, 9])]
df_table = generate_league_table_PRETTY(df_single)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,800.3 (727.8 - 872.8),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),2737.5 (2079.0 - 3396.1),nan (nan - nan),nan (nan - nan)
1,ASAQ term,768.1 (695.4 - 840.8),32.2 (-70.5 - 134.9),456.1 (332.3 - 579.9),456.1 (332.3 - 579.9),14.2 (Dominant - 121.9),3063.8 (2438.5 - 3689.1),326.3 (-581.9 - 1234.4),10.1 (Dominant - 116.1)
2,DP term,565.1 (447.5 - 682.7),203.0 (64.8 - 341.2),1271.2 (1143.9 - 1398.5),815.1 (637.5 - 992.7),4.0 (2.2 - 12.2),3433.0 (2926.1 - 3939.9),369.2 (-435.8 - 1174.1),1.8 (Dominant - 9.0)
3,ASAQ month,480.8 (341.6 - 620.0),84.2 (-97.9 - 266.3),1745.1 (1202.0 - 2288.1),473.9 (-83.9 - 1031.7),5.6 (Dominant - 57.8),3717.6 (3017.1 - 4418.2),284.6 (-580.1 - 1149.3),3.4 (Dominant - 51.1)
4,DP month,359.0 (243.2 - 474.8),121.8 (-59.2 - 302.8),4992.3 (4434.6 - 5550.1),3247.3 (2468.8 - 4025.7),26.7 (Dominant - 254.3),6733.8 (6073.0 - 7394.6),3016.2 (2053.1 - 3979.2),24.8 (Dominant - 242.7)


In [19]:
df_single = df[(df["archetype"] == "Southern") & (df["transmission_level"] == 0.1)]
df_single = df_single[df_single["scenario_number"].isin([0, 7, 8, 9])]
df_table = generate_league_table_PRETTY(df_single)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,800.3 (727.8 - 872.8),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),2737.5 (2079.0 - 3396.1),nan (nan - nan),nan (nan - nan)
1,DP term,565.1 (447.5 - 682.7),235.3 (97.1 - 373.4),1271.2 (1143.9 - 1398.5),1271.2 (1143.9 - 1398.5),5.4 (3.4 - 13.2),3433.0 (2926.1 - 3939.9),695.5 (-135.6 - 1526.5),3.0 (Dominant - 9.6)
2,ASAQ month,480.8 (341.6 - 620.0),84.2 (-97.9 - 266.3),1745.1 (1202.0 - 2288.1),473.9 (-83.9 - 1031.7),5.6 (Dominant - 58.9),3717.6 (3017.1 - 4418.2),284.6 (-580.1 - 1149.3),3.4 (Dominant - 52.3)
3,DP month,359.0 (243.2 - 474.8),121.8 (-59.2 - 302.8),4992.3 (4434.6 - 5550.1),3247.3 (2468.8 - 4025.7),26.7 (Dominant - 263.7),6733.8 (6073.0 - 7394.6),3016.2 (2053.1 - 3979.2),24.8 (Dominant - 240.8)


## 20% pfpr

In [20]:
df_single = df[(df["archetype"] == "Southern") & (df["transmission_level"] == 0.2)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 7, 8, 9])]
df_table = generate_league_table_PRETTY(df_single)
df_table.to_csv("league_tables/league_table_southern_pfpr20_all.csv")
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,998.6 (959.0 - 1038.2),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),3664.5 (2699.8 - 4629.2),nan (nan - nan),nan (nan - nan)
1,ASAQ term,939.8 (896.3 - 983.3),58.8 (-0.1 - 117.7),448.0 (326.1 - 569.9),448.0 (326.1 - 569.9),7.6 (Dominant - 47.9),3918.9 (3010.8 - 4826.9),254.4 (-1070.5 - 1579.3),4.3 (Dominant - 62.9)
2,DP term,747.9 (692.4 - 803.4),191.9 (121.4 - 262.4),1252.2 (1125.2 - 1379.1),804.2 (628.2 - 980.2),4.2 (2.8 - 6.9),4230.8 (3485.2 - 4976.4),311.9 (-863.1 - 1486.9),1.6 (Dominant - 8.6)
3,ASAQ month,673.5 (617.2 - 729.8),74.4 (-4.6 - 153.4),1719.3 (1184.2 - 2254.4),467.2 (-82.8 - 1017.1),6.3 (Dominant - 46.9),4485.5 (3634.4 - 5336.6),254.7 (-876.8 - 1386.2),3.4 (Dominant - 49.1)
4,DP month,513.6 (454.6 - 572.6),160.0 (78.4 - 241.5),4918.7 (4365.9 - 5471.4),3199.3 (2430.0 - 3968.7),20.0 (12.2 - 42.1),7352.7 (6573.1 - 8132.3),2867.2 (1713.0 - 4021.4),17.9 (9.3 - 39.2)


In [21]:
df_single = df[(df["archetype"] == "Southern") & (df["transmission_level"] == 0.2)]
df_single = df_single[df_single["scenario_number"].isin([0, 7, 8, 9])]
df_table = generate_league_table_PRETTY(df_single)
df_table.to_csv("league_tables/league_table_southern_pfpr20_reduced.csv")
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,998.6 (959.0 - 1038.2),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),3664.5 (2699.8 - 4629.2),nan (nan - nan),nan (nan - nan)
1,DP term,747.9 (692.4 - 803.4),250.7 (182.6 - 318.9),1252.2 (1125.2 - 1379.1),1252.2 (1125.2 - 1379.1),5.0 (3.8 - 6.9),4230.8 (3485.2 - 4976.4),566.3 (-653.0 - 1785.6),2.3 (Dominant - 7.6)
2,ASAQ month,673.5 (617.2 - 729.8),74.4 (-4.6 - 153.4),1719.3 (1184.2 - 2254.4),467.2 (-82.8 - 1017.1),6.3 (Dominant - 45.3),4485.5 (3634.4 - 5336.6),254.7 (-876.8 - 1386.2),3.4 (Dominant - 47.8)
3,DP month,513.6 (454.6 - 572.6),160.0 (78.4 - 241.5),4918.7 (4365.9 - 5471.4),3199.3 (2430.0 - 3968.7),20.0 (12.2 - 41.9),7352.7 (6573.1 - 8132.3),2867.2 (1713.0 - 4021.4),17.9 (9.3 - 39.1)


## 30% pfpr

In [22]:
df_single = df[(df["archetype"] == "Southern") & (df["transmission_level"] == 0.3)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 7, 8, 9])]
df_table = generate_league_table_PRETTY(df_single)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,1249.3 (1215.0 - 1283.6),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),4746.6 (3414.3 - 6078.9),nan (nan - nan),nan (nan - nan)
1,ASAQ term,1168.5 (1128.3 - 1208.7),80.8 (28.0 - 133.5),465.5 (338.9 - 592.1),465.5 (338.9 - 592.1),5.8 (3.1 - 16.5),4974.1 (3713.5 - 6234.7),227.5 (-1606.7 - 2061.7),2.8 (Dominant - 34.6)
2,DP term,941.0 (905.3 - 976.7),227.5 (173.8 - 281.1),1300.4 (1168.9 - 1431.9),834.9 (652.4 - 1017.4),3.7 (2.7 - 5.1),5212.6 (4155.6 - 6269.7),238.5 (-1406.6 - 1883.7),1.0 (Dominant - 8.6)
3,ASAQ month,877.2 (843.3 - 911.1),63.8 (14.6 - 113.0),1781.1 (1226.7 - 2335.5),480.7 (-89.1 - 1050.5),7.5 (Dominant - 34.3),5491.0 (4363.5 - 6618.5),278.4 (-1267.1 - 1823.8),4.4 (Dominant - 46.8)
4,DP month,718.7 (676.6 - 760.8),158.5 (104.4 - 212.7),5096.9 (4525.3 - 5668.5),3315.8 (2519.5 - 4112.1),20.9 (14.0 - 33.4),8448.5 (7420.1 - 9476.9),2957.5 (1431.5 - 4483.6),18.7 (8.6 - 33.6)


In [23]:
df_single = df[(df["archetype"] == "Southern") & (df["transmission_level"] == 0.3)]
df_single = df_single[df_single["scenario_number"].isin([0, 7, 8, 9])]
df_table = generate_league_table_PRETTY(df_single)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,1249.3 (1215.0 - 1283.6),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),4746.6 (3414.3 - 6078.9),nan (nan - nan),nan (nan - nan)
1,DP term,941.0 (905.3 - 976.7),308.2 (258.8 - 357.7),1300.4 (1168.9 - 1431.9),1300.4 (1168.9 - 1431.9),4.2 (3.5 - 5.1),5212.6 (4155.6 - 6269.7),466.0 (-1234.7 - 2166.7),1.5 (Dominant - 7.2)
2,ASAQ month,877.2 (843.3 - 911.1),63.8 (14.6 - 113.0),1781.1 (1226.7 - 2335.5),480.7 (-89.1 - 1050.5),7.5 (Dominant - 34.1),5491.0 (4363.5 - 6618.5),278.4 (-1267.1 - 1823.8),4.4 (Dominant - 46.4)
3,DP month,718.7 (676.6 - 760.8),158.5 (104.4 - 212.7),5096.9 (4525.3 - 5668.5),3315.8 (2519.5 - 4112.1),20.9 (14.0 - 33.3),8448.5 (7420.1 - 9476.9),2957.5 (1431.5 - 4483.6),18.7 (8.5 - 33.6)


## 40% pfpr

In [24]:
df_single = df[(df["archetype"] == "Southern") & (df["transmission_level"] == 0.4)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 7, 8, 9])]
df_table = generate_league_table_PRETTY(df_single)
df_table.to_csv("league_tables/league_table_southern_pfpr40_all.csv")
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,1316.3 (1276.7 - 1355.9),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),5169.8 (3689.1 - 6650.4),nan (nan - nan),nan (nan - nan)
1,ASAQ term,1222.6 (1191.8 - 1253.4),93.7 (43.5 - 143.8),448.7 (326.8 - 570.7),448.7 (326.8 - 570.7),4.8 (2.8 - 10.6),5326.2 (3941.4 - 6710.9),156.4 (-1870.9 - 2183.7),1.7 (Dominant - 28.1)
2,DP term,985.4 (950.1 - 1020.7),237.2 (190.4 - 284.0),1253.8 (1127.0 - 1380.6),805.1 (629.1 - 981.0),3.4 (2.5 - 4.6),5515.9 (4339.1 - 6692.8),189.7 (-1627.5 - 2007.0),0.8 (Dominant - 8.6)
3,ASAQ month,930.6 (891.4 - 969.8),54.8 (2.0 - 107.6),1716.1 (1182.6 - 2249.7),462.3 (-86.1 - 1010.8),8.4 (Dominant - 54.9),5808.3 (4576.5 - 7040.2),292.4 (-1411.2 - 1996.1),5.3 (Dominant - 80.3)
4,DP month,796.0 (759.5 - 832.5),134.6 (81.0 - 188.1),4912.3 (4361.9 - 5462.6),3196.1 (2429.6 - 3962.7),23.8 (15.4 - 41.0),8696.3 (7551.3 - 9841.4),2888.0 (1206.1 - 4569.8),21.5 (8.5 - 42.5)


In [25]:
df_single = df[(df["archetype"] == "Southern") & (df["transmission_level"] == 0.4)]
df_single = df_single[df_single["scenario_number"].isin([0, 7, 8, 9])]
df_table = generate_league_table_PRETTY(df_single)
df_table.to_csv("league_tables/league_table_southern_pfpr40_reduced.csv")
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,1316.3 (1276.7 - 1355.9),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),5169.8 (3689.1 - 6650.4),nan (nan - nan),nan (nan - nan)
1,DP term,985.4 (950.1 - 1020.7),330.9 (277.9 - 383.9),1253.8 (1127.0 - 1380.6),1253.8 (1127.0 - 1380.6),3.8 (3.2 - 4.6),5515.9 (4339.1 - 6692.8),346.2 (-1545.2 - 2237.5),1.0 (Dominant - 6.9)
2,ASAQ month,930.6 (891.4 - 969.8),54.8 (2.0 - 107.6),1716.1 (1182.6 - 2249.7),462.3 (-86.1 - 1010.8),8.4 (Dominant - 54.0),5808.3 (4576.5 - 7040.2),292.4 (-1411.2 - 1996.1),5.3 (Dominant - 81.1)
3,DP month,796.0 (759.5 - 832.5),134.6 (81.0 - 188.1),4912.3 (4361.9 - 5462.6),3196.1 (2429.6 - 3962.7),23.8 (15.4 - 41.0),8696.3 (7551.3 - 9841.4),2888.0 (1206.1 - 4569.8),21.5 (8.5 - 42.3)


# All league tables for Central

## 5% pfpr

In [26]:
df_single = df[(df["archetype"] == "Central") & (df["transmission_level"] == 0.05)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 7, 8, 9])]
df_table = generate_league_table_PRETTY(df_single)
df_table.to_csv("league_tables/league_table_central_pfpr05_all.csv", index=False)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,1205.9 (1139.3 - 1272.5),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),3221.5 (2393.6 - 4049.5),nan (nan - nan),nan (nan - nan)
1,ASAQ term,1063.3 (963.3 - 1163.3),142.6 (22.4 - 262.8),468.6 (341.2 - 596.1),468.6 (341.2 - 596.1),3.3 (1.5 - 15.4),3394.1 (2642.3 - 4146.0),172.6 (-945.8 - 1291.0),1.2 (Dominant - 16.4)
2,DP term,748.0 (647.1 - 848.9),315.3 (173.2 - 457.5),1305.3 (1170.3 - 1440.3),836.6 (651.0 - 1022.3),2.7 (1.7 - 5.0),3670.5 (3100.7 - 4240.4),276.4 (-667.0 - 1219.8),0.9 (Dominant - 4.5)
3,ASAQ month,580.1 (414.5 - 745.7),167.8 (-26.1 - 361.7),1647.6 (1136.6 - 2158.6),342.3 (-186.3 - 870.8),2.0 (Dominant - 17.8),3638.9 (2936.4 - 4341.3),-31.7 (-936.2 - 872.9),-0.2 (Dominant - 14.2)
4,DP month,295.8 (142.9 - 448.7),284.3 (59.0 - 509.7),4713.4 (4185.9 - 5241.0),3065.8 (2331.4 - 3800.3),10.8 (5.4 - 43.3),6238.7 (5623.4 - 6854.0),2599.8 (1666.0 - 3533.6),9.1 (4.2 - 36.6)


In [27]:
df_single = df[(df["archetype"] == "Central") & (df["transmission_level"] == 0.05)]
df_single = df_single[df_single["scenario_number"].isin([0, 7, 9])]
df_table = generate_league_table_PRETTY(df_single)
df_table.to_csv("league_tables/league_table_central_pfpr05_reduced.csv", index=False)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,1205.9 (1139.3 - 1272.5),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),3221.5 (2393.6 - 4049.5),nan (nan - nan),nan (nan - nan)
1,ASAQ month,580.1 (414.5 - 745.7),625.8 (447.3 - 804.2),1647.6 (1136.6 - 2158.6),1647.6 (1136.6 - 2158.6),2.6 (1.7 - 4.0),3638.9 (2936.4 - 4341.3),417.3 (-668.5 - 1503.1),0.7 (Dominant - 2.5)
2,DP month,295.8 (142.9 - 448.7),284.3 (59.0 - 509.7),4713.4 (4185.9 - 5241.0),3065.8 (2331.4 - 3800.3),10.8 (5.4 - 43.9),6238.7 (5623.4 - 6854.0),2599.8 (1666.0 - 3533.6),9.1 (4.2 - 37.7)


## 10% pfpr

In [28]:
df_single = df[(df["archetype"] == "Central") & (df["transmission_level"] == 0.1)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 7, 8, 9])]
df_table = generate_league_table_PRETTY(df_single)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,1187.8 (1127.4 - 1248.2),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),3303.2 (2450.9 - 4155.4),nan (nan - nan),nan (nan - nan)
1,ASAQ term,1054.8 (983.3 - 1126.3),133.0 (39.4 - 226.5),440.6 (320.9 - 560.2),440.6 (320.9 - 560.2),3.3 (1.8 - 10.9),3446.9 (2680.7 - 4213.2),143.8 (-1002.3 - 1289.9),1.1 (Dominant - 14.2)
2,DP term,730.3 (621.1 - 839.5),324.6 (194.1 - 455.1),1230.1 (1105.8 - 1354.4),789.5 (617.0 - 962.1),2.4 (1.6 - 4.2),3636.7 (3044.7 - 4228.7),189.7 (-778.6 - 1158.0),0.6 (Dominant - 4.0)
3,ASAQ month,569.0 (443.6 - 694.4),161.3 (-5.0 - 327.6),1542.0 (1064.8 - 2019.2),311.9 (-181.2 - 805.0),1.9 (Dominant - 15.0),3565.1 (2905.2 - 4224.9),-71.6 (-958.1 - 814.9),-0.4 (Dominant - 11.6)
4,DP month,306.1 (187.1 - 425.1),262.9 (90.0 - 435.9),4409.2 (3916.8 - 4901.6),2867.2 (2181.5 - 3552.9),10.9 (6.1 - 31.8),5990.7 (5413.2 - 6568.1),2425.6 (1548.8 - 3302.5),9.2 (4.7 - 27.3)


In [29]:
df_single = df[(df["archetype"] == "Central") & (df["transmission_level"] == 0.1)]
df_single = df_single[df_single["scenario_number"].isin([0, 7, 9])]
df_table = generate_league_table_PRETTY(df_single)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,1187.8 (1127.4 - 1248.2),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),3303.2 (2450.9 - 4155.4),nan (nan - nan),nan (nan - nan)
1,ASAQ month,569.0 (443.6 - 694.4),618.8 (479.6 - 758.1),1542.0 (1064.8 - 2019.2),1542.0 (1064.8 - 2019.2),2.5 (1.6 - 3.6),3565.1 (2905.2 - 4224.9),261.9 (-816.0 - 1339.8),0.4 (Dominant - 2.2)
2,DP month,306.1 (187.1 - 425.1),262.9 (90.0 - 435.9),4409.2 (3916.8 - 4901.6),2867.2 (2181.5 - 3552.9),10.9 (6.1 - 31.4),5990.7 (5413.2 - 6568.1),2425.6 (1548.8 - 3302.5),9.2 (4.6 - 26.9)


## 20% pfpr

In [30]:
df_single = df[(df["archetype"] == "Central") & (df["transmission_level"] == 0.2)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 7, 8, 9])]
df_table = generate_league_table_PRETTY(df_single)
df_table.to_csv("league_tables/league_table_central_pfpr20_all.csv", index=False)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,1336.9 (1296.5 - 1377.3),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),4230.3 (3071.7 - 5388.9),nan (nan - nan),nan (nan - nan)
1,ASAQ term,1243.2 (1198.5 - 1287.9),93.7 (33.5 - 153.9),446.1 (324.9 - 567.3),446.1 (324.9 - 567.3),4.8 (2.6 - 13.3),4379.4 (3314.6 - 5444.2),149.1 (-1424.5 - 1722.6),1.6 (Dominant - 24.4)
2,DP term,925.5 (879.2 - 971.8),317.7 (253.4 - 381.9),1245.0 (1119.2 - 1370.8),798.9 (624.2 - 973.5),2.5 (1.9 - 3.4),4476.0 (3646.5 - 5305.4),96.6 (-1253.1 - 1446.3),0.3 (Dominant - 4.7)
3,ASAQ month,848.1 (783.6 - 912.6),77.4 (-2.0 - 156.8),1563.5 (1079.2 - 2047.8),318.5 (-181.8 - 818.8),4.1 (Dominant - 31.8),4391.9 (3547.9 - 5235.9),-84.1 (-1267.4 - 1099.3),-1.1 (Dominant - 32.9)
4,DP month,556.8 (488.6 - 625.0),291.3 (197.4 - 385.2),4468.6 (3967.5 - 4969.8),2905.2 (2208.3 - 3602.0),10.0 (6.7 - 15.5),6733.1 (6032.0 - 7434.2),2341.2 (1243.9 - 3438.4),8.0 (4.0 - 13.9)


In [31]:
df_single = df[(df["archetype"] == "Central") & (df["transmission_level"] == 0.2)]
df_single = df_single[df_single["scenario_number"].isin([0, 7,8, 9])]
df_table = generate_league_table_PRETTY(df_single)
# df_table.to_csv("league_tables/league_table_central_pfpr20_reduced.csv", index=False)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,1336.9 (1296.5 - 1377.3),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),4230.3 (3071.7 - 5388.9),nan (nan - nan),nan (nan - nan)
1,DP term,925.5 (879.2 - 971.8),411.4 (349.9 - 472.8),1245.0 (1119.2 - 1370.8),1245.0 (1119.2 - 1370.8),3.0 (2.5 - 3.7),4476.0 (3646.5 - 5305.4),245.7 (-1179.2 - 1670.6),0.6 (Dominant - 4.1)
2,ASAQ month,848.1 (783.6 - 912.6),77.4 (-2.0 - 156.8),1563.5 (1079.2 - 2047.8),318.5 (-181.8 - 818.8),4.1 (Dominant - 31.3),4391.9 (3547.9 - 5235.9),-84.1 (-1267.4 - 1099.3),-1.1 (Dominant - 31.7)
3,DP month,556.8 (488.6 - 625.0),291.3 (197.4 - 385.2),4468.6 (3967.5 - 4969.8),2905.2 (2208.3 - 3602.0),10.0 (6.7 - 15.5),6733.1 (6032.0 - 7434.2),2341.2 (1243.9 - 3438.4),8.0 (4.1 - 13.8)


In [32]:
df_single = df[(df["archetype"] == "Central") & (df["transmission_level"] == 0.2)]
df_single = df_single[df_single["scenario_number"].isin([0, 7, 9])]
df_table = generate_league_table_PRETTY(df_single)
df_table.to_csv("league_tables/league_table_central_pfpr20_reduced.csv", index=False)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,1336.9 (1296.5 - 1377.3),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),4230.3 (3071.7 - 5388.9),nan (nan - nan),nan (nan - nan)
1,ASAQ month,848.1 (783.6 - 912.6),488.8 (412.6 - 564.9),1563.5 (1079.2 - 2047.8),1563.5 (1079.2 - 2047.8),3.2 (2.2 - 4.4),4391.9 (3547.9 - 5235.9),161.6 (-1271.8 - 1595.0),0.3 (Dominant - 3.3)
2,DP month,556.8 (488.6 - 625.0),291.3 (197.4 - 385.2),4468.6 (3967.5 - 4969.8),2905.2 (2208.3 - 3602.0),10.0 (6.7 - 15.5),6733.1 (6032.0 - 7434.2),2341.2 (1243.9 - 3438.4),8.0 (4.1 - 13.9)


## 30% pfpr

In [33]:
df_single = df[(df["archetype"] == "Central") & (df["transmission_level"] == 0.3)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 7, 8, 9])]
df_table = generate_league_table_PRETTY(df_single)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,1395.0 (1360.1 - 1429.9),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),4826.8 (3466.1 - 6187.4),nan (nan - nan),nan (nan - nan)
1,ASAQ term,1304.2 (1268.7 - 1339.7),90.9 (41.1 - 140.6),460.7 (335.6 - 585.7),460.7 (335.6 - 585.7),5.1 (2.9 - 11.6),4988.6 (3724.2 - 6253.1),161.8 (-1695.6 - 2019.3),1.8 (Dominant - 27.0)
2,DP term,990.5 (954.6 - 1026.4),313.6 (263.2 - 364.0),1285.0 (1155.4 - 1414.6),824.3 (644.2 - 1004.4),2.6 (2.0 - 3.4),5087.9 (4069.2 - 6106.5),99.3 (-1524.4 - 1723.0),0.3 (Dominant - 5.6)
3,ASAQ month,936.8 (889.8 - 983.8),53.7 (-5.3 - 112.8),1612.5 (1113.4 - 2111.5),327.5 (-188.1 - 843.1),6.1 (Dominant - 51.4),4994.1 (3988.6 - 5999.7),-93.8 (-1525.1 - 1337.6),-1.7 (Dominant - 63.5)
4,DP month,675.8 (634.6 - 717.0),261.0 (198.6 - 323.5),4604.5 (4089.5 - 5119.4),2992.0 (2274.9 - 3709.1),11.5 (8.1 - 16.2),7428.8 (6581.2 - 8276.4),2434.7 (1119.5 - 3749.8),9.3 (4.2 - 15.6)


## 40% pfpr

In [34]:
df_single = df[(df["archetype"] == "Central") & (df["transmission_level"] == 0.4)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 7, 8, 9])]
df_table = generate_league_table_PRETTY(df_single)
df_table.to_csv("league_tables/league_table_central_pfpr40_all.csv", index=False)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,1432.0 (1399.1 - 1464.9),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),5479.9 (3893.4 - 7066.4),nan (nan - nan),nan (nan - nan)
1,ASAQ term,1335.8 (1302.5 - 1369.1),96.1 (49.4 - 142.9),460.1 (335.0 - 585.2),460.1 (335.0 - 585.2),4.8 (2.9 - 9.6),5660.1 (4165.7 - 7154.6),180.2 (-1999.3 - 2359.7),1.9 (Dominant - 28.6)
2,DP term,1048.8 (1013.9 - 1083.7),287.1 (238.8 - 335.3),1283.6 (1153.9 - 1413.2),823.5 (643.3 - 1003.6),2.9 (2.1 - 3.8),5764.4 (4513.8 - 7015.0),104.3 (-1844.4 - 2053.0),0.4 (Dominant - 7.2)
3,ASAQ month,996.7 (963.2 - 1030.2),52.1 (3.7 - 100.5),1612.2 (1113.1 - 2111.4),328.6 (-187.1 - 844.4),6.3 (Dominant - 42.3),5726.2 (4503.9 - 6948.6),-38.2 (-1786.9 - 1710.6),-0.7 (Dominant - 64.3)
4,DP month,793.2 (757.3 - 829.1),203.4 (154.4 - 252.5),4607.2 (4091.5 - 5123.0),2995.0 (2277.2 - 3712.7),14.7 (10.4 - 20.9),8213.0 (7140.2 - 9285.8),2486.8 (860.4 - 4113.1),12.2 (4.1 - 21.8)


In [35]:
df_single = df[(df["archetype"] == "Central") & (df["transmission_level"] == 0.4)]
df_single = df_single[df_single["scenario_number"].isin([0, 7, 9])]
df_table = generate_league_table_PRETTY(df_single)
df_table.to_csv("league_tables/league_table_central_pfpr40_reduced.csv", index=False)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,1432.0 (1399.1 - 1464.9),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),5479.9 (3893.4 - 7066.4),nan (nan - nan),nan (nan - nan)
1,ASAQ month,996.7 (963.2 - 1030.2),435.3 (388.4 - 482.2),1612.2 (1113.1 - 2111.4),1612.2 (1113.1 - 2111.4),3.7 (2.5 - 5.0),5726.2 (4503.9 - 6948.6),246.3 (-1756.4 - 2249.1),0.6 (Dominant - 5.2)
2,DP month,793.2 (757.3 - 829.1),203.4 (154.4 - 252.5),4607.2 (4091.5 - 5123.0),2995.0 (2277.2 - 3712.7),14.7 (10.4 - 20.9),8213.0 (7140.2 - 9285.8),2486.8 (860.4 - 4113.1),12.2 (4.1 - 21.8)


# All league tables for Sahel

In [36]:
df_single[["scenario_number", "comments"]]

,scenario_number,comments
4,0,No IPTsc
19,7,ASAQ month
29,9,DP month


## 5% pfpr

In [37]:
df_single = df[(df["archetype"] == "Sahel") & (df["transmission_level"] == 0.05)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 7, 8, 9, 30, 31, 32, 33])]
df_table = generate_league_table_PRETTY(df_single)
df_table.to_csv("league_tables/league_table_sahel_pfpr05_all.csv", index=False)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,534.5 (433.2 - 635.8),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),2813.7 (2270.3 - 3357.1),nan (nan - nan),nan (nan - nan)
1,u10 SMC - SPAQ,445.3 (350.2 - 540.4),89.2 (-49.7 - 228.1),508.8 (216.2 - 801.5),508.8 (216.2 - 801.5),5.7 (Dominant - 55.8),3134.2 (2569.5 - 3699.0),320.5 (-463.2 - 1104.2),3.6 (Dominant - 46.7)
2,SPAQ term,434.0 (334.2 - 533.8),11.3 (-126.5 - 149.0),387.9 (262.1 - 513.6),-121.0 (-439.5 - 197.5),-10.7 (Dominant - 36.5),2971.0 (2475.7 - 3466.4),-163.2 (-914.4 - 588.0),-14.5 (Dominant - 71.3)
3,u10 SMC - DP,399.5 (308.4 - 490.6),34.5 (-100.5 - 169.5),1147.0 (852.5 - 1441.5),759.1 (438.9 - 1079.3),22.0 (Dominant - 159.8),3688.1 (3139.6 - 4236.6),717.1 (-22.0 - 1456.2),20.8 (Dominant - 146.0)
4,u15 SMC - SPAQ,380.0 (277.9 - 482.1),19.6 (-117.3 - 156.4),897.9 (381.7 - 1414.1),-249.0 (-843.3 - 345.3),-12.7 (Dominant - 68.6),3386.0 (2699.6 - 4072.4),-302.1 (-1180.8 - 576.6),-15.4 (Dominant - 94.1)
5,DP term,292.7 (174.1 - 411.3),87.3 (-69.3 - 243.9),1296.9 (1166.2 - 1427.5),398.9 (-133.5 - 931.4),4.6 (Dominant - 50.8),3564.3 (3137.9 - 3990.7),178.3 (-629.8 - 986.4),2.0 (Dominant - 41.0)
6,SPAQ month,280.2 (160.4 - 400.0),12.5 (-156.2 - 181.1),1122.7 (708.6 - 1536.7),-174.2 (-608.4 - 260.0),-14.0 (Dominant - 41.7),3374.6 (2794.3 - 3955.0),-189.7 (-909.9 - 530.5),-15.2 (Dominant - 61.0)
7,u15 SMC - DP,275.3 (184.6 - 366.0),4.9 (-145.3 - 155.2),2017.7 (1501.0 - 2534.4),895.1 (232.9 - 1557.2),180.9 (Dominant - 184.6),4321.8 (3671.4 - 4972.2),947.2 (75.4 - 1818.9),191.4 (Dominant - 195.9)
8,DP month,184.9 (41.4 - 328.4),90.4 (-79.3 - 260.1),3858.6 (3428.5 - 4288.7),1840.9 (1168.6 - 2513.2),20.4 (Dominant - 222.8),5900.2 (5326.5 - 6473.8),1578.4 (711.1 - 2445.6),17.5 (Dominant - 181.0)


In [38]:
df_single = df[(df["archetype"] == "Sahel") & (df["transmission_level"] == 0.05)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 7, 9, 31, 33])]
df_table = generate_league_table_PRETTY(df_single)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,534.5 (433.2 - 635.8),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),2813.7 (2270.3 - 3357.1),nan (nan - nan),nan (nan - nan)
1,SPAQ term,434.0 (334.2 - 533.8),100.5 (-41.6 - 242.6),387.9 (262.1 - 513.6),387.9 (262.1 - 513.6),3.9 (Dominant - 35.3),2971.0 (2475.7 - 3466.4),157.3 (-578.0 - 892.6),1.6 (Dominant - 29.5)
2,u15 SMC - SPAQ,380.0 (277.9 - 482.1),54.1 (-88.7 - 196.8),897.9 (381.7 - 1414.1),510.1 (-21.2 - 1041.4),9.4 (Dominant - 86.8),3386.0 (2699.6 - 4072.4),415.0 (-431.5 - 1261.5),7.7 (Dominant - 83.9)
3,SPAQ month,280.2 (160.4 - 400.0),99.8 (-57.7 - 257.2),1122.7 (708.6 - 1536.7),224.7 (-437.0 - 886.5),2.3 (Dominant - 32.1),3374.6 (2794.3 - 3955.0),-11.4 (-910.3 - 887.5),-0.1 (Dominant - 33.2)
4,u15 SMC - DP,275.3 (184.6 - 366.0),4.9 (-145.3 - 155.2),2017.7 (1501.0 - 2534.4),895.1 (232.9 - 1557.2),180.9 (Dominant - 188.9),4321.8 (3671.4 - 4972.2),947.2 (75.4 - 1818.9),191.4 (Dominant - 197.5)
5,DP month,184.9 (41.4 - 328.4),90.4 (-79.3 - 260.1),3858.6 (3428.5 - 4288.7),1840.9 (1168.6 - 2513.2),20.4 (Dominant - 206.1),5900.2 (5326.5 - 6473.8),1578.4 (711.1 - 2445.6),17.5 (Dominant - 186.5)


In [39]:
df_single = df[(df["archetype"] == "Sahel") & (df["transmission_level"] == 0.05)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 7, 9, 31, 33])]
df_table = generate_league_table_PRETTY(df_single)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,534.5 (433.2 - 635.8),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),2813.7 (2270.3 - 3357.1),nan (nan - nan),nan (nan - nan)
1,SPAQ term,434.0 (334.2 - 533.8),100.5 (-41.6 - 242.6),387.9 (262.1 - 513.6),387.9 (262.1 - 513.6),3.9 (Dominant - 35.5),2971.0 (2475.7 - 3466.4),157.3 (-578.0 - 892.6),1.6 (Dominant - 30.9)
2,u15 SMC - SPAQ,380.0 (277.9 - 482.1),54.1 (-88.7 - 196.8),897.9 (381.7 - 1414.1),510.1 (-21.2 - 1041.4),9.4 (Dominant - 89.0),3386.0 (2699.6 - 4072.4),415.0 (-431.5 - 1261.5),7.7 (Dominant - 85.2)
3,SPAQ month,280.2 (160.4 - 400.0),99.8 (-57.7 - 257.2),1122.7 (708.6 - 1536.7),224.7 (-437.0 - 886.5),2.3 (Dominant - 33.2),3374.6 (2794.3 - 3955.0),-11.4 (-910.3 - 887.5),-0.1 (Dominant - 33.9)
4,u15 SMC - DP,275.3 (184.6 - 366.0),4.9 (-145.3 - 155.2),2017.7 (1501.0 - 2534.4),895.1 (232.9 - 1557.2),180.9 (Dominant - 184.0),4321.8 (3671.4 - 4972.2),947.2 (75.4 - 1818.9),191.4 (Dominant - 197.7)
5,DP month,184.9 (41.4 - 328.4),90.4 (-79.3 - 260.1),3858.6 (3428.5 - 4288.7),1840.9 (1168.6 - 2513.2),20.4 (Dominant - 202.9),5900.2 (5326.5 - 6473.8),1578.4 (711.1 - 2445.6),17.5 (Dominant - 178.6)


In [40]:
df_single = df[(df["archetype"] == "Sahel") & (df["transmission_level"] == 0.05)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 7, 9, 33])]
df_table = generate_league_table_PRETTY(df_single)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,534.5 (433.2 - 635.8),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),2813.7 (2270.3 - 3357.1),nan (nan - nan),nan (nan - nan)
1,SPAQ term,434.0 (334.2 - 533.8),100.5 (-41.6 - 242.6),387.9 (262.1 - 513.6),387.9 (262.1 - 513.6),3.9 (Dominant - 36.7),2971.0 (2475.7 - 3466.4),157.3 (-578.0 - 892.6),1.6 (Dominant - 29.5)
2,SPAQ month,280.2 (160.4 - 400.0),153.8 (-2.0 - 309.7),1122.7 (708.6 - 1536.7),734.8 (302.1 - 1167.5),4.8 (Dominant - 31.5),3374.6 (2794.3 - 3955.0),403.6 (-359.4 - 1166.6),2.6 (Dominant - 21.0)
3,u15 SMC - DP,275.3 (184.6 - 366.0),4.9 (-145.3 - 155.2),2017.7 (1501.0 - 2534.4),895.1 (232.9 - 1557.2),180.9 (Dominant - 181.8),4321.8 (3671.4 - 4972.2),947.2 (75.4 - 1818.9),191.4 (Dominant - 202.0)
4,DP month,184.9 (41.4 - 328.4),90.4 (-79.3 - 260.1),3858.6 (3428.5 - 4288.7),1840.9 (1168.6 - 2513.2),20.4 (Dominant - 206.7),5900.2 (5326.5 - 6473.8),1578.4 (711.1 - 2445.6),17.5 (Dominant - 182.5)


In [41]:
df_single = df[(df["archetype"] == "Sahel") & (df["transmission_level"] == 0.05)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 7, 9])]
df_table = generate_league_table_PRETTY(df_single)
df_table.to_csv("league_tables/league_table_sahel_pfpr05_reduced.csv", index=False)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,534.5 (433.2 - 635.8),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),2813.7 (2270.3 - 3357.1),nan (nan - nan),nan (nan - nan)
1,SPAQ term,434.0 (334.2 - 533.8),100.5 (-41.6 - 242.6),387.9 (262.1 - 513.6),387.9 (262.1 - 513.6),3.9 (Dominant - 35.8),2971.0 (2475.7 - 3466.4),157.3 (-578.0 - 892.6),1.6 (Dominant - 29.7)
2,SPAQ month,280.2 (160.4 - 400.0),153.8 (-2.0 - 309.7),1122.7 (708.6 - 1536.7),734.8 (302.1 - 1167.5),4.8 (Dominant - 30.3),3374.6 (2794.3 - 3955.0),403.6 (-359.4 - 1166.6),2.6 (Dominant - 20.9)
3,DP month,184.9 (41.4 - 328.4),95.3 (-91.5 - 282.2),3858.6 (3428.5 - 4288.7),2735.9 (2138.9 - 3332.9),28.7 (Dominant - 294.5),5900.2 (5326.5 - 6473.8),2525.5 (1709.5 - 3341.6),26.5 (Dominant - 264.5)


## 10% pfpr

In [42]:
df_single = df[(df["archetype"] == "Sahel") & (df["transmission_level"] == 0.1)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 7, 8, 9, 30, 31, 32, 33])]
df_table = generate_league_table_PRETTY(df_single)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,740.1 (676.4 - 803.8),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),3434.1 (2710.8 - 4157.5),nan (nan - nan),nan (nan - nan)
1,u10 SMC - SPAQ,642.5 (577.2 - 707.8),97.6 (6.4 - 188.9),469.9 (199.9 - 739.9),469.9 (199.9 - 739.9),4.8 (1.1 - 27.5),3669.4 (2967.4 - 4371.3),235.2 (-772.7 - 1243.2),2.4 (Dominant - 27.0)
2,SPAQ term,626.8 (561.3 - 692.3),15.7 (-76.7 - 108.1),383.0 (258.6 - 507.5),-86.9 (-384.2 - 210.4),-5.5 (Dominant - 44.0),3513.2 (2873.3 - 4153.1),-156.1 (-1106.0 - 793.7),-9.9 (Dominant - 129.6)
3,u10 SMC - DP,588.0 (526.1 - 649.9),38.8 (-51.2 - 128.9),1056.4 (785.3 - 1327.4),673.3 (375.1 - 971.6),17.3 (Dominant - 171.7),4146.9 (3474.3 - 4819.6),633.7 (-294.7 - 1562.2),16.3 (Dominant - 165.7)
4,u15 SMC - SPAQ,543.5 (482.2 - 604.8),44.5 (-42.6 - 131.6),861.7 (366.6 - 1356.8),-194.7 (-759.1 - 369.8),-4.4 (Dominant - 56.2),3832.5 (3071.9 - 4593.2),-314.4 (-1329.8 - 701.0),-7.1 (Dominant - 98.1)
5,DP term,452.0 (364.4 - 539.6),91.5 (-15.5 - 198.4),1281.9 (1151.8 - 1412.0),420.2 (-91.7 - 932.2),4.6 (Dominant - 37.3),3963.8 (3448.4 - 4479.3),131.3 (-787.5 - 1050.2),1.4 (Dominant - 33.3)
6,SPAQ month,412.3 (307.6 - 517.0),39.7 (-96.8 - 176.3),1107.0 (698.9 - 1515.0),-175.0 (-603.2 - 253.3),-4.4 (Dominant - 42.6),3741.2 (3099.1 - 4383.3),-222.7 (-1046.1 - 600.7),-5.6 (Dominant - 70.6)
7,u15 SMC - DP,402.6 (334.6 - 470.6),9.7 (-115.2 - 134.5),1938.5 (1442.0 - 2435.0),831.5 (188.9 - 1474.1),85.9 (Dominant - 206.1),4645.7 (3941.4 - 5349.9),904.5 (-48.5 - 1857.5),93.5 (Dominant - 229.8)
8,DP month,289.8 (189.3 - 390.3),112.8 (-8.5 - 234.2),3797.6 (3375.8 - 4219.4),1859.2 (1207.7 - 2510.6),16.5 (Dominant - 114.7),6124.8 (5539.7 - 6710.0),1479.2 (563.6 - 2394.8),13.1 (Dominant - 95.8)


In [43]:
df_single = df[(df["archetype"] == "Sahel") & (df["transmission_level"] == 0.1)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 7, 9, 31, 33])]
df_table = generate_league_table_PRETTY(df_single)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,740.1 (676.4 - 803.8),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),3434.1 (2710.8 - 4157.5),nan (nan - nan),nan (nan - nan)
1,SPAQ term,626.8 (561.3 - 692.3),113.3 (22.0 - 204.6),383.0 (258.6 - 507.5),383.0 (258.6 - 507.5),3.4 (1.6 - 14.4),3513.2 (2873.3 - 4153.1),79.1 (-886.7 - 1044.9),0.7 (Dominant - 15.3)
2,u15 SMC - SPAQ,543.5 (482.2 - 604.8),83.3 (-6.3 - 173.0),861.7 (366.6 - 1356.8),478.7 (-31.8 - 989.2),5.7 (Dominant - 43.8),3832.5 (3071.9 - 4593.2),319.3 (-674.7 - 1313.4),3.8 (Dominant - 42.1)
3,SPAQ month,412.3 (307.6 - 517.0),131.2 (9.8 - 252.6),1107.0 (698.9 - 1515.0),245.3 (-396.3 - 886.8),1.9 (Dominant - 15.5),3741.2 (3099.1 - 4383.3),-91.4 (-1086.8 - 904.1),-0.7 (Dominant - 13.2)
4,u15 SMC - DP,402.6 (334.6 - 470.6),9.7 (-115.2 - 134.5),1938.5 (1442.0 - 2435.0),831.5 (188.9 - 1474.1),85.9 (Dominant - 207.7),4645.7 (3941.4 - 5349.9),904.5 (-48.5 - 1857.5),93.5 (Dominant - 220.0)
5,DP month,289.8 (189.3 - 390.3),112.8 (-8.5 - 234.2),3797.6 (3375.8 - 4219.4),1859.2 (1207.7 - 2510.6),16.5 (Dominant - 117.8),6124.8 (5539.7 - 6710.0),1479.2 (563.6 - 2394.8),13.1 (Dominant - 96.3)


In [44]:
df_single = df[(df["archetype"] == "Sahel") & (df["transmission_level"] == 0.1)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 7, 9])]
df_table = generate_league_table_PRETTY(df_single)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,740.1 (676.4 - 803.8),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),3434.1 (2710.8 - 4157.5),nan (nan - nan),nan (nan - nan)
1,SPAQ term,626.8 (561.3 - 692.3),113.3 (22.0 - 204.6),383.0 (258.6 - 507.5),383.0 (258.6 - 507.5),3.4 (1.6 - 14.2),3513.2 (2873.3 - 4153.1),79.1 (-886.7 - 1044.9),0.7 (Dominant - 15.2)
2,SPAQ month,412.3 (307.6 - 517.0),214.5 (91.0 - 338.0),1107.0 (698.9 - 1515.0),723.9 (297.3 - 1150.5),3.4 (1.3 - 8.8),3741.2 (3099.1 - 4383.3),228.0 (-678.6 - 1134.5),1.1 (Dominant - 6.7)
3,DP month,289.8 (189.3 - 390.3),122.5 (-22.7 - 267.7),3797.6 (3375.8 - 4219.4),2690.7 (2103.8 - 3277.5),22.0 (Dominant - 181.7),6124.8 (5539.7 - 6710.0),2383.7 (1514.9 - 3252.4),19.5 (Dominant - 153.6)


## 20% pfpr

In [45]:
df_single = df[(df["archetype"] == "Sahel") & (df["transmission_level"] == 0.2)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 7, 8, 9, 30, 31, 32, 33])]
df_table = generate_league_table_PRETTY(df_single)
df_table.to_csv("league_tables/league_table_sahel_pfpr20_all.csv", index=False)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,906.9 (861.2 - 952.6),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),4215.1 (3242.5 - 5187.7),nan (nan - nan),nan (nan - nan)
1,u10 SMC - SPAQ,798.5 (751.5 - 845.5),108.4 (42.9 - 173.9),456.1 (193.5 - 718.7),456.1 (193.5 - 718.7),4.2 (1.6 - 11.6),4389.5 (3471.8 - 5307.3),174.4 (-1162.8 - 1511.7),1.6 (Dominant - 18.1)
2,SPAQ term,793.9 (751.2 - 836.6),4.7 (-58.8 - 68.1),381.8 (258.0 - 505.6),-74.3 (-364.6 - 216.0),-15.8 (Dominant - 65.2),4233.3 (3373.6 - 5092.9),-156.3 (-1413.8 - 1101.2),-33.3 (Dominant - 261.3)
3,u10 SMC - DP,736.3 (686.7 - 785.9),57.6 (-7.9 - 123.0),1025.2 (761.5 - 1288.9),643.4 (352.0 - 934.7),11.2 (Dominant - 83.9),4826.9 (3947.4 - 5706.5),593.7 (-636.2 - 1823.5),10.3 (Dominant - 101.3)
4,u15 SMC - SPAQ,696.6 (646.2 - 747.0),39.7 (-31.0 - 110.4),859.9 (365.1 - 1354.7),-165.3 (-726.0 - 395.4),-4.2 (Dominant - 59.0),4538.5 (3600.5 - 5476.5),-288.4 (-1574.2 - 997.5),-7.3 (Dominant - 128.7)
5,DP term,630.7 (579.2 - 682.2),65.9 (-6.2 - 138.0),1273.7 (1143.7 - 1403.6),413.8 (-97.8 - 925.3),6.3 (Dominant - 49.3),4665.5 (3950.3 - 5380.6),126.9 (-1052.6 - 1306.4),1.9 (Dominant - 49.9)
6,SPAQ month,592.8 (543.0 - 642.6),37.9 (-33.8 - 109.6),1099.8 (693.6 - 1506.1),-173.8 (-600.4 - 252.7),-4.6 (Dominant - 53.8),4438.4 (3639.2 - 5237.5),-227.1 (-1299.6 - 845.4),-6.0 (Dominant - 111.3)
7,u15 SMC - DP,565.4 (512.7 - 618.1),27.4 (-45.1 - 99.9),1934.3 (1438.8 - 2429.9),834.5 (193.7 - 1475.3),30.5 (Dominant - 276.0),5341.0 (4476.2 - 6205.7),902.6 (-274.9 - 2080.1),33.0 (Dominant - 332.1)
8,DP month,485.1 (426.5 - 543.7),80.3 (1.5 - 159.2),3776.7 (3356.4 - 4197.0),1842.4 (1192.6 - 2492.2),22.9 (7.4 - 138.8),6809.1 (6084.3 - 7534.0),1468.2 (339.8 - 2596.5),18.3 (Dominant - 113.3)


In [46]:
df_single = df[(df["archetype"] == "Sahel") & (df["transmission_level"] == 0.2)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 7, 9, 31, 33])]
df_table = generate_league_table_PRETTY(df_single)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,906.9 (861.2 - 952.6),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),4215.1 (3242.5 - 5187.7),nan (nan - nan),nan (nan - nan)
1,SPAQ term,793.9 (751.2 - 836.6),113.1 (50.6 - 175.6),381.8 (258.0 - 505.6),381.8 (258.0 - 505.6),3.4 (1.9 - 7.8),4233.3 (3373.6 - 5092.9),18.2 (-1279.9 - 1316.2),0.2 (Dominant - 13.9)
2,u15 SMC - SPAQ,696.6 (646.2 - 747.0),97.3 (31.3 - 163.3),859.9 (365.1 - 1354.7),478.1 (-32.0 - 988.2),4.9 (Dominant - 17.9),4538.5 (3600.5 - 5476.5),305.3 (-967.1 - 1577.6),3.1 (Dominant - 23.7)
3,SPAQ month,592.8 (543.0 - 642.6),103.8 (33.0 - 174.6),1099.8 (693.6 - 1506.1),239.9 (-400.3 - 880.1),2.3 (Dominant - 13.0),4438.4 (3639.2 - 5237.5),-100.2 (-1332.4 - 1132.1),-1.0 (Dominant - 14.4)
4,u15 SMC - DP,565.4 (512.7 - 618.1),27.4 (-45.1 - 99.9),1934.3 (1438.8 - 2429.9),834.5 (193.7 - 1475.3),30.5 (Dominant - 275.8),5341.0 (4476.2 - 6205.7),902.6 (-274.9 - 2080.1),33.0 (Dominant - 321.3)
5,DP month,485.1 (426.5 - 543.7),80.3 (1.5 - 159.2),3776.7 (3356.4 - 4197.0),1842.4 (1192.6 - 2492.2),22.9 (7.2 - 140.4),6809.1 (6084.3 - 7534.0),1468.2 (339.8 - 2596.5),18.3 (Dominant - 114.4)


In [47]:
df_single = df[(df["archetype"] == "Sahel") & (df["transmission_level"] == 0.2)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 7, 9])]
df_table = generate_league_table_PRETTY(df_single)
df_table.to_csv("league_tables/league_table_sahel_pfpr20_reduced.csv", index=False)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,906.9 (861.2 - 952.6),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),4215.1 (3242.5 - 5187.7),nan (nan - nan),nan (nan - nan)
1,SPAQ term,793.9 (751.2 - 836.6),113.1 (50.6 - 175.6),381.8 (258.0 - 505.6),381.8 (258.0 - 505.6),3.4 (1.9 - 7.8),4233.3 (3373.6 - 5092.9),18.2 (-1279.9 - 1316.2),0.2 (Dominant - 13.9)
2,SPAQ month,592.8 (543.0 - 642.6),201.1 (135.5 - 266.6),1099.8 (693.6 - 1506.1),718.0 (293.3 - 1142.7),3.6 (1.4 - 6.6),4438.4 (3639.2 - 5237.5),205.1 (-968.7 - 1378.9),1.0 (Dominant - 7.4)
3,DP month,485.1 (426.5 - 543.7),107.7 (30.8 - 184.6),3776.7 (3356.4 - 4197.0),2676.9 (2092.4 - 3261.5),24.8 (13.5 - 81.0),6809.1 (6084.3 - 7534.0),2370.8 (1291.8 - 3449.7),22.0 (9.6 - 75.5)


## 30% pfpr

In [48]:
df_single = df[(df["archetype"] == "Sahel") & (df["transmission_level"] == 0.3)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 7, 8, 9, 30, 31, 32, 33])]
df_table = generate_league_table_PRETTY(df_single)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,1036.3 (988.5 - 1084.1),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),4870.3 (3678.5 - 6062.0),nan (nan - nan),nan (nan - nan)
1,u10 SMC - SPAQ,905.7 (863.8 - 947.6),130.6 (66.9 - 194.3),448.2 (190.5 - 705.9),448.2 (190.5 - 705.9),3.4 (1.3 - 7.7),4982.6 (3871.5 - 6093.8),112.4 (-1517.0 - 1741.8),0.9 (Dominant - 15.4)
2,SPAQ term,883.5 (840.6 - 926.4),22.2 (-37.8 - 82.3),368.3 (248.6 - 487.9),-79.9 (-364.0 - 204.2),-3.6 (Dominant - 51.9),4750.9 (3711.4 - 5790.3),-231.8 (-1753.3 - 1289.7),-10.4 (Dominant - 257.8)
3,u10 SMC - DP,843.5 (798.4 - 888.6),39.9 (-22.3 - 102.1),1007.7 (748.9 - 1266.4),639.4 (354.3 - 924.4),16.0 (Dominant - 162.8),5401.7 (4336.7 - 6466.7),650.8 (-837.4 - 2139.0),16.3 (Dominant - 204.5)
4,u15 SMC - SPAQ,814.1 (772.4 - 855.8),29.4 (-32.0 - 90.9),824.3 (350.2 - 1298.4),-183.3 (-723.4 - 356.8),-6.2 (Dominant - 84.3),5122.8 (4015.5 - 6230.2),-278.9 (-1815.2 - 1257.5),-9.5 (Dominant - 209.0)
5,DP term,726.9 (682.0 - 771.8),87.3 (26.0 - 148.5),1232.6 (1107.2 - 1358.1),408.3 (-82.1 - 898.7),4.7 (Dominant - 18.6),5163.1 (4272.0 - 6054.2),40.3 (-1381.1 - 1461.6),0.5 (Dominant - 23.5)
6,SPAQ month,700.0 (653.2 - 746.8),26.9 (-38.0 - 91.7),1063.8 (671.5 - 1456.1),-168.8 (-580.7 - 243.0),-6.3 (Dominant - 75.0),4933.1 (3984.3 - 5881.8),-230.0 (-1531.6 - 1071.6),-8.6 (Dominant - 192.1)
7,u15 SMC - DP,695.9 (655.7 - 736.1),4.1 (-57.6 - 65.8),1854.2 (1379.2 - 2329.1),790.4 (174.4 - 1406.4),194.5 (Dominant - 392.3),5900.2 (4869.1 - 6931.3),967.1 (-434.1 - 2368.3),238.0 (Dominant - 517.5)
8,DP month,594.1 (540.6 - 647.6),101.8 (34.9 - 168.7),3653.7 (3250.0 - 4057.4),1799.6 (1176.2 - 2422.9),17.7 (9.1 - 52.4),7217.2 (6355.9 - 8078.5),1317.0 (-26.4 - 2660.5),12.9 (Dominant - 44.7)


In [49]:
df_single = df[(df["archetype"] == "Sahel") & (df["transmission_level"] == 0.3)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 7, 9, 31, 33])]
df_table = generate_league_table_PRETTY(df_single)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,1036.3 (988.5 - 1084.1),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),4870.3 (3678.5 - 6062.0),nan (nan - nan),nan (nan - nan)
1,SPAQ term,883.5 (840.6 - 926.4),152.8 (88.5 - 217.1),368.3 (248.6 - 487.9),368.3 (248.6 - 487.9),2.4 (1.4 - 4.4),4750.9 (3711.4 - 5790.3),-119.4 (-1700.8 - 1462.0),-0.8 (Dominant - 10.4)
2,u15 SMC - SPAQ,814.1 (772.4 - 855.8),69.3 (9.5 - 129.2),824.3 (350.2 - 1298.4),456.1 (-32.9 - 945.0),6.6 (Dominant - 35.2),5122.8 (4015.5 - 6230.2),372.0 (-1146.8 - 1890.7),5.4 (Dominant - 52.7)
3,SPAQ month,700.0 (653.2 - 746.8),114.1 (51.4 - 176.9),1063.8 (671.5 - 1456.1),239.5 (-375.9 - 854.8),2.1 (Dominant - 9.7),4933.1 (3984.3 - 5881.8),-189.8 (-1648.0 - 1268.4),-1.7 (Dominant - 12.9)
4,u15 SMC - DP,695.9 (655.7 - 736.1),4.1 (-57.6 - 65.8),1854.2 (1379.2 - 2329.1),790.4 (174.4 - 1406.4),194.5 (Dominant - 372.9),5900.2 (4869.1 - 6931.3),967.1 (-434.1 - 2368.3),238.0 (Dominant - 514.9)
5,DP month,594.1 (540.6 - 647.6),101.8 (34.9 - 168.7),3653.7 (3250.0 - 4057.4),1799.6 (1176.2 - 2422.9),17.7 (9.0 - 52.0),7217.2 (6355.9 - 8078.5),1317.0 (-26.4 - 2660.5),12.9 (Dominant - 45.2)


In [50]:
df_single = df[(df["archetype"] == "Sahel") & (df["transmission_level"] == 0.3)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 7, 9])]
df_table = generate_league_table_PRETTY(df_single)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,1036.3 (988.5 - 1084.1),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),4870.3 (3678.5 - 6062.0),nan (nan - nan),nan (nan - nan)
1,SPAQ term,883.5 (840.6 - 926.4),152.8 (88.5 - 217.1),368.3 (248.6 - 487.9),368.3 (248.6 - 487.9),2.4 (1.4 - 4.4),4750.9 (3711.4 - 5790.3),-119.4 (-1700.8 - 1462.0),-0.8 (Dominant - 10.5)
2,SPAQ month,700.0 (653.2 - 746.8),183.5 (119.9 - 247.0),1063.8 (671.5 - 1456.1),695.5 (285.4 - 1105.6),3.8 (1.5 - 7.1),4933.1 (3984.3 - 5881.8),182.2 (-1225.1 - 1589.5),1.0 (Dominant - 9.3)
3,DP month,594.1 (540.6 - 647.6),105.9 (34.8 - 177.0),3653.7 (3250.0 - 4057.4),2589.9 (2027.0 - 3152.9),24.5 (13.7 - 73.1),7217.2 (6355.9 - 8078.5),2284.2 (1002.8 - 3565.6),21.6 (8.1 - 68.6)


## 40% pfpr

In [51]:
df_single = df[(df["archetype"] == "Sahel") & (df["transmission_level"] == 0.4)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 7, 8, 9, 30, 31, 32, 33])]
df_table = generate_league_table_PRETTY(df_single)
df_table.to_csv("league_tables/league_table_sahel_pfpr40_all.csv", index=False)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,1156.1 (1117.1 - 1195.1),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),5576.2 (4145.9 - 7006.6),nan (nan - nan),nan (nan - nan)
1,u10 SMC - SPAQ,1010.1 (972.1 - 1048.1),146.1 (91.6 - 200.6),486.7 (206.8 - 766.5),486.7 (206.8 - 766.5),3.3 (1.3 - 6.4),5687.5 (4356.9 - 7018.1),111.3 (-1842.3 - 2064.9),0.8 (Dominant - 15.2)
2,u10 SMC - DP,959.9 (915.6 - 1004.2),50.2 (-8.2 - 108.6),1095.6 (814.0 - 1377.2),609.0 (211.9 - 1006.0),12.1 (Dominant - 95.9),6193.1 (4896.8 - 7489.3),505.6 (-1352.1 - 2363.2),10.1 (Dominant - 137.1)
3,SPAQ term,956.0 (915.6 - 996.4),3.9 (-56.2 - 63.9),383.3 (258.8 - 507.8),-712.3 (-1020.2 - -404.4),-183.3 (Dominant - 364.7),5378.7 (4140.4 - 6616.9),-814.4 (-2607.0 - 978.2),-209.6 (Dominant - 516.3)
4,u15 SMC - SPAQ,928.5 (890.1 - 966.9),27.5 (-28.2 - 83.2),864.4 (367.6 - 1361.2),481.1 (-31.1 - 993.3),17.5 (Dominant - 170.9),5857.1 (4527.8 - 7186.4),478.4 (-1338.3 - 2295.1),17.4 (Dominant - 305.9)
5,u15 SMC - DP,825.7 (784.0 - 867.4),102.8 (46.2 - 159.5),1942.7 (1444.7 - 2440.7),1078.3 (374.8 - 1781.7),10.5 (3.4 - 26.6),6705.2 (5449.0 - 7961.3),848.1 (-980.8 - 2677.0),8.2 (Dominant - 34.2)
6,DP term,820.6 (779.0 - 862.2),5.2 (-53.7 - 64.0),1279.9 (1149.4 - 1410.4),-662.8 (-1177.6 - -148.0),-128.3 (Dominant - 344.8),5887.3 (4774.2 - 7000.4),-817.8 (-2496.2 - 860.5),-158.3 (Dominant - 523.0)
7,SPAQ month,788.7 (744.6 - 832.8),31.9 (-28.6 - 92.4),1108.2 (699.8 - 1516.6),-171.7 (-600.5 - 257.0),-5.4 (Dominant - 60.6),5628.8 (4478.6 - 6778.9),-258.6 (-1859.1 - 1342.0),-8.1 (Dominant - 200.2)
8,DP month,693.1 (648.0 - 738.2),95.6 (32.6 - 158.6),3803.4 (3379.7 - 4227.2),2695.3 (2106.7 - 3283.8),28.2 (15.9 - 81.7),8062.1 (6985.9 - 9138.4),2433.4 (858.2 - 4008.6),25.5 (8.0 - 80.2)


In [52]:
df_single = df[(df["archetype"] == "Sahel") & (df["transmission_level"] == 0.4)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 7, 8, 9, 30, 31])]
df_table = generate_league_table_PRETTY(df_single)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,1156.1 (1117.1 - 1195.1),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),5576.2 (4145.9 - 7006.6),nan (nan - nan),nan (nan - nan)
1,u10 SMC - SPAQ,1010.1 (972.1 - 1048.1),146.1 (91.6 - 200.6),486.7 (206.8 - 766.5),486.7 (206.8 - 766.5),3.3 (1.3 - 6.4),5687.5 (4356.9 - 7018.1),111.3 (-1842.3 - 2064.9),0.8 (Dominant - 15.3)
2,SPAQ term,956.0 (915.6 - 996.4),54.0 (-1.4 - 109.5),383.3 (258.8 - 507.8),-103.3 (-409.7 - 203.0),-1.9 (Dominant - 9.0),5378.7 (4140.4 - 6616.9),-308.8 (-2126.5 - 1508.8),-5.7 (Dominant - 65.9)
3,u15 SMC - SPAQ,928.5 (890.1 - 966.9),27.5 (-28.2 - 83.2),864.4 (367.6 - 1361.2),481.1 (-31.1 - 993.3),17.5 (Dominant - 182.2),5857.1 (4527.8 - 7186.4),478.4 (-1338.3 - 2295.1),17.4 (Dominant - 305.7)
4,DP term,820.6 (779.0 - 862.2),108.0 (51.5 - 164.5),1279.9 (1149.4 - 1410.4),415.5 (-98.2 - 929.2),3.8 (Dominant - 11.4),5887.3 (4774.2 - 7000.4),30.2 (-1703.6 - 1764.0),0.3 (Dominant - 19.1)
5,SPAQ month,788.7 (744.6 - 832.8),31.9 (-28.6 - 92.4),1108.2 (699.8 - 1516.6),-171.7 (-600.5 - 257.0),-5.4 (Dominant - 64.4),5628.8 (4478.6 - 6778.9),-258.6 (-1859.1 - 1342.0),-8.1 (Dominant - 199.8)
6,DP month,693.1 (648.0 - 738.2),95.6 (32.6 - 158.6),3803.4 (3379.7 - 4227.2),2695.3 (2106.7 - 3283.8),28.2 (15.9 - 81.8),8062.1 (6985.9 - 9138.4),2433.4 (858.2 - 4008.6),25.5 (7.9 - 80.1)


In [53]:
df_single = df[(df["archetype"] == "Sahel") & (df["transmission_level"] == 0.4)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 7, 9, 31])]
df_table = generate_league_table_PRETTY(df_single)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,1156.1 (1117.1 - 1195.1),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),5576.2 (4145.9 - 7006.6),nan (nan - nan),nan (nan - nan)
1,SPAQ term,956.0 (915.6 - 996.4),200.1 (143.9 - 256.4),383.3 (258.8 - 507.8),383.3 (258.8 - 507.8),1.9 (1.2 - 3.0),5378.7 (4140.4 - 6616.9),-197.6 (-2089.5 - 1694.3),-1.0 (Dominant - 8.8)
2,u15 SMC - SPAQ,928.5 (890.1 - 966.9),27.5 (-28.2 - 83.2),864.4 (367.6 - 1361.2),481.1 (-31.1 - 993.3),17.5 (Dominant - 182.2),5857.1 (4527.8 - 7186.4),478.4 (-1338.3 - 2295.1),17.4 (Dominant - 307.8)
3,SPAQ month,788.7 (744.6 - 832.8),139.9 (81.5 - 198.3),1108.2 (699.8 - 1516.6),243.8 (-399.4 - 886.9),1.7 (Dominant - 7.2),5628.8 (4478.6 - 6778.9),-228.3 (-1986.2 - 1529.5),-1.6 (Dominant - 12.0)
4,DP month,693.1 (648.0 - 738.2),95.6 (32.6 - 158.6),3803.4 (3379.7 - 4227.2),2695.3 (2106.7 - 3283.8),28.2 (15.9 - 81.5),8062.1 (6985.9 - 9138.4),2433.4 (858.2 - 4008.6),25.5 (8.0 - 79.1)


In [55]:
df_single = df[(df["archetype"] == "Sahel") & (df["transmission_level"] == 0.4)]
df_single = df_single[df_single["scenario_number"].isin([0, 6, 7, 9])]
df_table = generate_league_table_PRETTY(df_single)
df_table.to_csv("league_tables/league_table_sahel_pfpr40_reduced.csv", index=False)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,1156.1 (1117.1 - 1195.1),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),5576.2 (4145.9 - 7006.6),nan (nan - nan),nan (nan - nan)
1,SPAQ term,956.0 (915.6 - 996.4),200.1 (143.9 - 256.4),383.3 (258.8 - 507.8),383.3 (258.8 - 507.8),1.9 (1.2 - 3.0),5378.7 (4140.4 - 6616.9),-197.6 (-2089.5 - 1694.3),-1.0 (Dominant - 8.8)
2,SPAQ month,788.7 (744.6 - 832.8),167.4 (107.6 - 227.1),1108.2 (699.8 - 1516.6),724.8 (297.9 - 1151.8),4.3 (1.7 - 8.2),5628.8 (4478.6 - 6778.9),250.1 (-1439.9 - 1940.1),1.5 (Dominant - 12.6)
3,DP month,693.1 (648.0 - 738.2),95.6 (32.6 - 158.6),3803.4 (3379.7 - 4227.2),2695.3 (2106.7 - 3283.8),28.2 (15.9 - 81.8),8062.1 (6985.9 - 9138.4),2433.4 (858.2 - 4008.6),25.5 (7.9 - 80.1)


In [56]:
5

5

# How does higher-ITN compare?
Make league table with just scenarios 0 and 5 for each archetype and transmission level

In [62]:
# ITN cost for higher coverage: 20% higher coverage of 5000 people
total_cost = 0.2*5000*2.48/2 # Factor of 2 accounts for the fact that the IPTsc has to be done every year, while the ITN distribution was only done in the first year
# print(total_cost)
itn_cost_per_1k = total_cost/5
print(itn_cost_per_1k)

248.0


In [65]:
# Set cost_IPTsc_mean for scenario 5 to be itn_cost_per_1k
df.loc[(df["scenario_number"] == 5), "cost_IPTsc_mean"] = itn_cost_per_1k

In [66]:
df_single = df[(df["archetype"] == "Southern") & (df["scenario_number"].isin([0, 5])) & (df["transmission_level"] == 0.05)]
df_table = generate_league_table_PRETTY(df_single)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,663.5 (570.2 - 756.8),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),2351.6 (1816.3 - 2886.8),nan (nan - nan),nan (nan - nan)
1,No IPTsc - higher ITN,587.2 (462.7 - 711.7),76.3 (-79.2 - 231.8),248.0 (248.0 - 248.0),248.0 (248.0 - 248.0),3.2 (Dominant - 32.0),2323.3 (1861.5 - 2785.0),-28.3 (-735.3 - 678.6),-0.4 (Dominant - 35.9)


In [67]:
df_single = df[(df["archetype"] == "Southern") & (df["scenario_number"].isin([0, 5])) & (df["transmission_level"] == 0.2)]
df_table = generate_league_table_PRETTY(df_single)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,998.6 (959.0 - 1038.2),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),3664.5 (2699.8 - 4629.2),nan (nan - nan),nan (nan - nan)
1,No IPTsc - higher ITN,962.7 (917.8 - 1007.6),35.9 (-23.9 - 95.8),248.0 (248.0 - 248.0),248.0 (248.0 - 248.0),6.9 (Dominant - 70.8),3656.7 (2773.9 - 4539.5),-7.8 (-1315.4 - 1299.9),-0.2 (Dominant - 142.6)


In [68]:
df_single = df[(df["archetype"] == "Southern") & (df["scenario_number"].isin([0, 5])) & (df["transmission_level"] == 0.4)]
df_table = generate_league_table_PRETTY(df_single)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,1316.3 (1276.7 - 1355.9),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),5169.8 (3689.1 - 6650.4),nan (nan - nan),nan (nan - nan)
1,No IPTsc - higher ITN,1299.0 (1262.0 - 1336.0),17.3 (-37.0 - 71.5),248.0 (248.0 - 248.0),248.0 (248.0 - 248.0),14.4 (Dominant - 119.9),5226.2 (3810.3 - 6642.1),56.4 (-1992.3 - 2105.1),3.3 (Dominant - 395.9)


In [69]:
df_single = df[(df["archetype"] == "Central") & (df["scenario_number"].isin([0, 5])) & (df["transmission_level"] == 0.05)]
df_table = generate_league_table_PRETTY(df_single)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,1205.9 (1139.3 - 1272.5),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),3221.5 (2393.6 - 4049.5),nan (nan - nan),nan (nan - nan)
1,No IPTsc - higher ITN,1031.3 (915.5 - 1147.1),174.6 (40.9 - 308.3),248.0 (248.0 - 248.0),248.0 (248.0 - 248.0),1.4 (0.8 - 5.3),2952.0 (2276.6 - 3627.3),-269.6 (-1338.1 - 798.9),-1.5 (Dominant - 6.3)


In [70]:
df_single = df[(df["archetype"] == "Central") & (df["scenario_number"].isin([0, 5])) & (df["transmission_level"] == 0.2)]
df_table = generate_league_table_PRETTY(df_single)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,1336.9 (1296.5 - 1377.3),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),4230.3 (3071.7 - 5388.9),nan (nan - nan),nan (nan - nan)
1,No IPTsc - higher ITN,1308.6 (1266.7 - 1350.5),28.3 (-30.1 - 86.6),248.0 (248.0 - 248.0),248.0 (248.0 - 248.0),8.8 (Dominant - 84.3),4093.5 (3065.0 - 5122.0),-136.8 (-1686.0 - 1412.4),-4.8 (Dominant - 219.8)


In [71]:
df_single = df[(df["archetype"] == "Central") & (df["scenario_number"].isin([0, 5])) & (df["transmission_level"] == 0.4)]
df_table = generate_league_table_PRETTY(df_single)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,1432.0 (1399.1 - 1464.9),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),5479.9 (3893.4 - 7066.4),nan (nan - nan),nan (nan - nan)
1,No IPTsc - higher ITN,1415.8 (1385.8 - 1445.8),16.2 (-28.2 - 60.7),248.0 (248.0 - 248.0),248.0 (248.0 - 248.0),15.3 (Dominant - 145.9),5468.4 (3973.5 - 6963.2),-11.5 (-2191.3 - 2168.2),-0.7 (Dominant - 485.0)


In [72]:
df_single = df[(df["archetype"] == "Sahel") & (df["scenario_number"].isin([0, 5])) & (df["transmission_level"] == 0.05)]
df_table = generate_league_table_PRETTY(df_single)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,534.5 (433.2 - 635.8),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),2813.7 (2270.3 - 3357.1),nan (nan - nan),nan (nan - nan)
1,No IPTsc - higher ITN,422.3 (318.6 - 526.0),112.2 (-32.8 - 257.2),248.0 (248.0 - 248.0),248.0 (248.0 - 248.0),2.2 (Dominant - 19.2),2790.7 (2329.1 - 3252.2),-23.1 (-736.0 - 689.9),-0.2 (Dominant - 20.3)


In [73]:
df_single = df[(df["archetype"] == "Sahel") & (df["scenario_number"].isin([0, 5])) & (df["transmission_level"] == 0.2)]
df_table = generate_league_table_PRETTY(df_single)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,906.9 (861.2 - 952.6),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),4215.1 (3242.5 - 5187.7),nan (nan - nan),nan (nan - nan)
1,No IPTsc - higher ITN,840.2 (796.7 - 883.7),66.8 (3.7 - 129.8),248.0 (248.0 - 248.0),248.0 (248.0 - 248.0),3.7 (1.7 - 20.1),4201.9 (3318.4 - 5085.3),-13.2 (-1327.2 - 1300.7),-0.2 (Dominant - 39.5)


In [74]:
df_single = df[(df["archetype"] == "Sahel") & (df["scenario_number"].isin([0, 5])) & (df["transmission_level"] == 0.4)]
df_table = generate_league_table_PRETTY(df_single)
df_table

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,1156.1 (1117.1 - 1195.1),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),5576.2 (4145.9 - 7006.6),nan (nan - nan),nan (nan - nan)
1,No IPTsc - higher ITN,1096.0 (1053.7 - 1138.3),60.1 (2.4 - 117.8),248.0 (248.0 - 248.0),248.0 (248.0 - 248.0),4.1 (1.8 - 24.3),5602.1 (4250.4 - 6953.9),25.9 (-1942.2 - 1993.9),0.4 (Dominant - 67.9)


In [75]:
itn_funder_icers = np.array([3.2,6.9,14.4,1.4,8.8,15.3,2.2,3.7,4.1])
itn_hs_icers = np.array([-0.4,-0.2,3.3,-1.5,-4.8,-0.7,-0.2,-0.2,0.4])

In [76]:
itn_funder_icers.mean()

6.666666666666668

In [77]:
itn_hs_icers.mean()

-0.47777777777777786

In [79]:
np.median(itn_funder_icers)

4.1

In [80]:
np.median(itn_hs_icers)

-0.2

In [83]:
# Quick check
df_single = df[(df["archetype"] == "Southern") & (df["transmission_level"] == 0.2) & (df["scenario_number"].isin([0, 7,8,9]))]
df_single

,archetype,transmission_level,scenario_number,cost_mean,cost_std,cost_IPTsc_mean,cost_IPTsc_std,cases_mean,cases_std,comments
87,Southern,0.2,0,3664.479750,492.202836,0.000000,0.000000,998.606661,20.229171,No IPTsc
102,Southern,0.2,7,4485.500148,434.233769,1719.331447,273.010255,673.520540,28.706974,ASAQ month
107,Southern,0.2,8,4230.775705,380.415075,1252.164820,64.768735,747.883366,28.291114,DP term
112,Southern,0.2,9,7352.704375,397.738731,4918.650930,282.028193,513.554611,30.133000,DP month


In [84]:
generate_league_table_PRETTY(df_single)

C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_low"].iloc[i] = np.percentile(icers, 2.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table["icer_high"].iloc[i] = np.percentile(icers, 97.5)
C:\Users\joshsu\AppData\Local\Temp\ipykernel_35472\559688894.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,Scenario,Cases,Incremental Cases Averted,Cost (Funder Perspective),Incremental Cost (Funder Perspective),ICER (Funder Perspective),Cost (Health System Perspective),Incremental Cost (Health System Perspective),ICER (Health System Perspective)
0,No IPTsc,998.6 (959.0 - 1038.2),nan (nan - nan),0.0 (0.0 - 0.0),nan (nan - nan),nan (nan - nan),3664.5 (2699.8 - 4629.2),nan (nan - nan),nan (nan - nan)
1,DP term,747.9 (692.4 - 803.4),250.7 (182.6 - 318.9),1252.2 (1125.2 - 1379.1),1252.2 (1125.2 - 1379.1),5.0 (3.8 - 7.0),4230.8 (3485.2 - 4976.4),566.3 (-653.0 - 1785.6),2.3 (Dominant - 7.5)
2,ASAQ month,673.5 (617.2 - 729.8),74.4 (-4.6 - 153.4),1719.3 (1184.2 - 2254.4),467.2 (-82.8 - 1017.1),6.3 (Dominant - 46.1),4485.5 (3634.4 - 5336.6),254.7 (-876.8 - 1386.2),3.4 (Dominant - 48.0)
3,DP month,513.6 (454.6 - 572.6),160.0 (78.4 - 241.5),4918.7 (4365.9 - 5471.4),3199.3 (2430.0 - 3968.7),20.0 (12.1 - 42.0),7352.7 (6573.1 - 8132.3),2867.2 (1713.0 - 4021.4),17.9 (9.3 - 39.1)
